In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-04-28 23:49:36--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-04-28 23:49:36--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?fPulpWqefzcIDghdQqg_W6ShkVD_DTB-tq0v8TOAki-McmCbXnWs9ecHR-9CCaLl0D9YqPKy98LlKUmtP0R6pd-Blph7faq1AtRWYujCwD2-26mua

In [ ]:
!nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
nvcc fatal   : No input files specified; use option --help for more information


In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git 

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_9128d0w
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_9128d0w
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=699c6d860aaa4f74b918bbd24f407f0490bfa822a83fb187adf763299dcb90d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-gpqpxpn_/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin
from google.colab import drive
drive.mount('/content/drive/')

created output directory at /content/src
Out bin /content/result.out
Mounted at /content/drive/


In [79]:
%%cuda --name KERNELS.cu 

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime_api.h>
#include "device_launch_parameters.h"
#include <cusolverDn.h>
#include <cuda_runtime.h>

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/functionsV2.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/KERNELSH.h"

/* Kernel definitions */
__global__ void INPUT_UNROLLING(int stride, int Filter_Height,
                                float *Input, int H1, int W1, int D1,
                                float *X_unrolled, int H2, int W2, int D2,
                                int Output_Height, int Output_Width)
{  
    int bx = blockIdx.x, by = blockIdx.y, bz = blockIdx.z;
    int tx = threadIdx.x, ty = threadIdx.y;
 
    // Select row and column values 
    int row =  by * TileDW + ty;
    int col = bx * TileDW + tx;
    int depth = bz;
 
    int col_no_strided = col, row_no_strided = row;
    int depth_offset = depth * W2 * Filter_Height * Filter_Height;

    /* 
      Note for bx, by and bz= 0, stride = 2: 
          @ tx = 0, ty = 0 -> First multiply the col * stride, row * stride; = 0, 0
                            you are shifting in x direction using local col
                            you are shifting in y direction using local row;
          @ tx = 1, ty = 0 -> First multiply the col * stride, row * stride; = 2, 0 
                            you are shifting in x direction using local col
                            you are shifting in y direction using local row;   
          @ tx = 0, ty = 1 -> First multiply the col * stride, row * stride; = 0, 2 
                            you are shifting in x direction using local col
                            you are shifting in y direction using local row;                   
    */ 
  
    col *= stride; row *= stride;
 
    // Limit number of threads 
    if (row_no_strided < Output_Height && col_no_strided < Output_Width && depth < D1)
    {   
        // Each thread unrolls k x k elements
        for (int local_row = 0; local_row < Filter_Height; local_row++)
        {
            for (int local_col = 0; local_col < Filter_Height; local_col++)
            {                                  
                // 1. local row and column shifts affect the locations in Unrolled matrix
                // 2. For each col and row non strided values -> you are adding an offset to columns and rows in Unrolled matrix
                // 3. Offset the depth using "depth_offset" variable
                X_unrolled[local_col * W2 + local_row * Filter_Height * W2 + col_no_strided + row_no_strided * Output_Width + depth_offset] = 
                Input[(row + local_row) * W1 + (col + local_col) + depth * H1 * W1];
            }
        }
    }

}

__global__ void DWConv2d_kernel(float *Input, int H1, int W1, int D1,
                                float *Filter, int H2, int W2, int D2,
                                float *Output, int H3, int W3, int D3,
                                int stride)
{
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int bz = blockIdx.z;

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = by * TileDW + ty;
    int col = bx * TileDW + tx;
    int dep = bz;

    float Pvalue = 0;

    if (row < H3 && col < W3 && dep < D3)
    {
        // 1 thread unrolls kxk section
        for (int j = 0; j < H2; j++)
        {
            for (int i = 0; i < W2; i++)
            {
                Pvalue += Filter[j * W2 + i + dep * H2 * W2] *
                    Input[(j * W1 + row * stride * W1) + (i + col * stride) + dep * H1 * W1];
            }
        }
        Output[row * W3 + col + dep * H3 * W3] = Pvalue;
    }

}

__global__ void MatrixMulKernel(float *M, int H1, int W1, int D1,
                                float *N, int H2, int W2, int D2,
                                float *P, int H3, int W3, int D3,
                                int num_blocks, int activation, 
                                int IS_BIASED, float *bias_mat)
{
  __shared__ float Mds[Tile_GEMM][Tile_GEMM];
  __shared__ float Nds[Tile_GEMM][THREAD_GRANULARITY_BLOCKS * Tile_GEMM];

  int bx = blockIdx.x * THREAD_GRANULARITY_BLOCKS;
  int by = blockIdx.y;
  int tx = threadIdx.x;
  int ty = threadIdx.y;

  // Identify the row and column of the d_P element to work on
  int Row = by * Tile_GEMM + ty;
  int Col = bx * Tile_GEMM + tx;
  float Pvalue = 0;
  float Pvalue_2 = 0;

  // Loop over the d_M and d_N tiles required to compute d_P element
  for (int ph = 0; ph < num_blocks; ++ph)
  {
    // Collaborative loading of d_M and d_N tiles into shared memory
    if ((Row < H1) && (ph * Tile_GEMM + tx) < W1)
    {
      Mds[ty][tx] = M[Row * W1 + ph * Tile_GEMM + tx];
    }

    if ((ph * Tile_GEMM + ty) < H2 && Col < W2)
    {
      Nds[ty][tx] = N[(ph * Tile_GEMM + ty) * W2 + Col];
    }

    if ((ph * Tile_GEMM + ty) < H2 && Col + Tile_GEMM < W2)
    {
      Nds[ty][tx + Tile_GEMM] = N[(ph * Tile_GEMM + ty) * W2 + Col + Tile_GEMM];
    }     
   
    __syncthreads();

    for (int k = 0; k < Tile_GEMM && (ph * Tile_GEMM) + k < W1; ++k)
    {
      Pvalue += Mds[ty][k] * Nds[k][tx];
    }

    if (Col + Tile_GEMM < W2)
    {
      for (int k = 0; k < Tile_GEMM && (ph * Tile_GEMM) + k < W1; ++k)
      {
        Pvalue_2 += Mds[ty][k] * Nds[k][tx + Tile_GEMM];
      } 
    }
    __syncthreads();

  }

  if ((Row < H1) && (Col < W2))
  {
    P[Row * W3 + Col] = Pvalue;
    
    switch (IS_BIASED) 
    {
      case BIASED:
        Pvalue = Pvalue + bias_mat[Row];
        break;
      
      default:
        break;
    } 
            
    switch (activation) 
    {
      case SWISH_ACTIVATION:
        // Swish activation function
        P[Row * W3 + Col] = Pvalue / (1 + __expf(-1 * Pvalue));
        break;

      case SIGMOID_ACTIVATION:
        // Sigmoid activation function
        P[Row * W3 + Col] = 1 / (1 + __expf(-1 * Pvalue));
        break;

      default:
        break;
    }
  }

  if ((Row < H1) && (Col + Tile_GEMM < W2))
    {
      P[Row * W3 + Col + Tile_GEMM] = Pvalue_2;
      
      switch (IS_BIASED) 
      {
        case BIASED:
          Pvalue_2 = Pvalue_2 + bias_mat[Row];
          break;
        
        default:
          break;
      } 
              
      switch (activation) 
      {
        case SWISH_ACTIVATION:
          // Swish activation function
          P[Row * W3 + Col + Tile_GEMM] = Pvalue_2 / (1 + __expf(-1 * Pvalue_2));
          break;

        case SIGMOID_ACTIVATION:
          // Sigmoid activation function
          P[Row * W3 + Col + Tile_GEMM] = 1 / (1 + __expf(-1 * Pvalue_2));
          break;

        default:
          break;
      }
    }    
}


__global__ void ConvChannelElementWiseMultiplication(float *A, int H1, int W1, int D1,
                                                     float *B)
{
    int row = blockIdx.y * Tile_GEMM + threadIdx.y;
    int col = blockIdx.x * Tile_GEMM + threadIdx.x;
    int depth = blockIdx.z;

    int index = depth * W1 * H1 + row * W1 + col;

    if ((row < H1) && (col < W1) && (depth < D1))
    {
        A[index] = A[index] * B[depth];
    }
}

// Used with MBConv layers that has skip identity = true
__global__ void Identity_Skip(float *A,  int H1, int W1, int D1,
                              float *B)
{
    int row = blockIdx.y * Tile_GEMM + threadIdx.y;
    int col = blockIdx.x * Tile_GEMM + threadIdx.x;
    int depth = blockIdx.z;

    int index = depth * W1 * H1 + row * W1 + col;

    if ((row < H1) && (col < W1) && (depth < D1))
    {
        A[index] = A[index] + B[index];
    }
}

__global__ void Complete_Padding_Process(float *Original_Padded, int H1, int W1, int D1, 
                                         float *Original,        int H2, int W2, int D2,
                                         int padding_value)
{   
    // There must be a constant shift between indeces in 2 matrices
    // The code is based on x axis only
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int depth = blockIdx.z * blockDim.z + threadIdx.z;

    int index = depth * W2 * H2 + row * W2 + col;
    int Padding_Index = depth * W1 * H1 + (row + padding_value) * W1 + (col + padding_value);

    if ((row < (H2)) && (col < (W2)) && (depth < (D2)))
    {
        Original_Padded[Padding_Index] = Original[index];
    }
}

/* Batch Normalization Kernels */
const int BLOCK_SIZE = 16;

__global__ void BN_Kernel_Mean_Reduction(float *input, int H1, int W1, int D1,
                                         float *Mean, int W2)
{
    /*
        This code works on 2 * Block_Size elements.
        i.e. for 512 Block_Size -> we are reducing 1024 elements.
        Each thread loads 2 elements, one at tx and the
        other shifted by blockIdx.x.
    */

    __shared__ float partialSum[2 * BLOCK_SIZE];
    float tmp = 0;

    unsigned int tx = threadIdx.x;
    int bx = blockDim.x;

    int by_index = blockIdx.y;
    int bx_index = blockIdx.x;

    // The start variable is to get offset for input matrix in loading
    unsigned int start = blockIdx.x * (2 * blockDim.x);
    int start_yDir = blockIdx.y * W1;

    if (start + tx < W1 && start_yDir < H1 * W1)
        // Load 2 elements in the shared memory
        partialSum[tx] = input[start + tx + start_yDir];
    else
        partialSum[tx] = tmp;

    if (tx + bx + start < W1 && start_yDir < H1 * W1)
        partialSum[bx + tx] = input[start + bx + tx + start_yDir];
    else
        partialSum[bx + tx] = tmp;


    unsigned int stride = 0;

    __syncthreads();

    for (stride = blockDim.x; stride > 0; stride = stride / 2)
    {
        __syncthreads();
        if (tx < stride)
            partialSum[tx] += partialSum[tx + stride];
    }

    __syncthreads();


    if (tx == 0)
        Mean[bx_index + by_index * W2] = partialSum[tx];

}

__global__ void ElementWiseSquaring(float *A, int H1, int W1, int D1)
{
    int row = blockIdx.y * Tile_GEMM + threadIdx.y;
    int col = blockIdx.x * Tile_GEMM + threadIdx.x;
    int depth = blockIdx.z;

    int index = depth * W1 * H1 + row * W1 + col;

    if ((row < H1) && (col < W1) && (depth < D1))
    {
        A[index] = A[index] * A[index];
    }
}

__global__ void ElementWiseSubtraction(float *A, int H1, int W1, int D1,
                                       float *mean)
{
    int row = blockIdx.y * Tile_GEMM + threadIdx.y;
    int col = blockIdx.x * Tile_GEMM + threadIdx.x;
    int depth = blockIdx.z;

    int index = depth * W1 * H1 + row * W1 + col;

    if ((row < H1) && (col < W1) && (depth < D1))
    {
        A[index] = A[index] - mean[depth];
    }
}


__global__ void BN_Kernel_Final_Layer(float *A, int H1, int W1, int D1, 
                                      float *D_mean, float *D_variance,
                                      float *D_weight, float *D_bias,
                                      int activate)
{
    // Activate values are assigned as follow
    /*
      0 -> no activation, 1 -> swish, 2 -> sigmoid
    */
    int row = blockIdx.y * Tile_GEMM + threadIdx.y;
    int col = blockIdx.x * Tile_GEMM + threadIdx.x;
    int depth = blockIdx.z;

    int index = depth * W1 * H1 + row * W1 + col;
    int index3 = depth;

    float tmp = 0;
 
    if ((row < H1) && (col < W1) && (depth < D1))
    {
        A[index] = ((A[index] - D_mean[index3]) / (sqrtf(D_variance[index3] + 0.001))) * D_weight[index3] + D_bias[index3];
        tmp = A[index];

        switch (activate) {
                  case 1:
                      // Swish activation function
                      A[index] = tmp / (1 + __expf(-1 * tmp));
                      break;
                  case 2:
                      // Sigmoid activation function
                      A[index] = 1 / (1 + __expf(-1 * tmp));
                      break;
                  default:
                      break;
                    }
    }
}


'File written in /content/src/KERNELS.cu'

In [80]:
%%cuda --name APP.cu 

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime_api.h>
#include "device_launch_parameters.h"
#include <cusolverDn.h>
#include <cuda_runtime.h>


#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/Input_For_Stem_Layer.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/Stem/Stem_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/functionsV2.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/CONFIG.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/Input_Matrix.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/KERNELSH.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MBConv1_0/MBConv1_0_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MBConv1_0/MBConv1_0_project_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MBConv1_0/MBConv1_0_squeeze_excitation_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_1/MBConv6_1_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_1/MBConv6_1_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_1/MBConv6_1_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_1/MBConv6_1_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_2/MBConv6_2_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_2/MBConv6_2_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_2/MBConv6_2_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_2/MBConv6_2_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_3/MBConv6_3_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_3/MBConv6_3_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_3/MBConv6_3_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_3/MBConv6_3_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_4/MBConv6_4_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_4/MBConv6_4_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_4/MBConv6_4_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_4/MBConv6_4_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_5/MBConv6_5_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_5/MBConv6_5_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_5/MBConv6_5_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_5/MBConv6_5_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_6/MBConv6_6_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_6/MBConv6_6_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_6/MBConv6_6_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_6/MBConv6_6_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_7/MBConv6_7_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_7/MBConv6_7_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_7/MBConv6_7_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_7/MBConv6_7_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_8/MBConv6_8_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_8/MBConv6_8_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_8/MBConv6_8_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_8/MBConv6_8_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_9/MBConv6_9_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_9/MBConv6_9_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_9/MBConv6_9_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_9/MBConv6_9_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_10/MBConv6_10_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_10/MBConv6_10_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_10/MBConv6_10_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_10/MBConv6_10_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_11/MBConv6_11_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_11/MBConv6_11_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_11/MBConv6_11_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_11/MBConv6_11_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_12/MBConv6_12_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_12/MBConv6_12_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_12/MBConv6_12_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_12/MBConv6_12_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_13/MBConv6_13_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_13/MBConv6_13_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_13/MBConv6_13_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_13/MBConv6_13_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_14/MBConv6_14_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_14/MBConv6_14_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_14/MBConv6_14_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_14/MBConv6_14_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_15/MBConv6_15_expansion_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_15/MBConv6_15_depthwise_conv_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_15/MBConv6_15_squeeze_excitation_parameters.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/MbConv6_15/MBConv6_15_project_conv_parameters.h"

#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/Head/Head_conv_parameters.h"


int MBCONV1_0_flag = 0;

int main()
{
  // 1. Define dimensions for input image.
  set_allocate_copy_array_Device(&DInput_Mat, Input_for_stem_conv,
                                 INPUT_IMAGE_HEIGHT, INPUT_IMAGE_WIDTH, 
                                 INPUT_IMAGE_DEPTH,
                                 "Input Image is allocated in device memory");  

  // 2. Get layers' filters ready
  set_allocate_copy_array_Device(&F_STEM, Stem_conv2d_weights,
                                 STEM_FILTER_HEIGHT, STEM_FILTER_WIDTH, 
                                 STEM_FILTER_DEPTH * STEM_FILTER_DENSITY,
                                 "Stem Filter  is allocated in device memory");
  
  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_1_0_EXPD_WEIGHTS, NULL, 
                            MBCONV_1_0_EXPD_F_HEIGHT,   MBCONV_1_0_EXPD_F_WIDTH, 
                            MBCONV_1_0_EXPD_F_DEPTH * MBCONV_1_0_EXPD_F_DENSITY,
                            &D_MBConv_1_0_DW_WEIGHTS, MBConv1_0_depthwise_conv_conv2d_weights, 
                            MBCONV_1_0_DW_F_HEIGHT, MBCONV_1_0_DW_F_WIDTH, 
                            MBCONV_1_0_DW_F_DEPTH * MBCONV_1_0_DW_F_DENSITY,
                            &D_MBConv_1_0_SQZ_1_WEIGHTS, MBConv1_0_squeeze_excitation1_conv2d_weights,
                            MBCONV_1_0_SQZ_1_F_HEIGHT, MBCONV_1_0_SQZ_1_F_WIDTH, 
                            MBCONV_1_0_SQZ_1_F_DEPTH * MBCONV_1_0_SQZ_1_F_DENSITY,
                            &D_MBConv_1_0_SQZ_2_WEIGHTS, MBConv1_0_squeeze_excitation2_conv2d_weights, 
                            MBCONV_1_0_SQZ_2_F_HEIGHT, MBCONV_1_0_SQZ_2_F_WIDTH, 
                            MBCONV_1_0_SQZ_2_F_DEPTH * MBCONV_1_0_SQZ_2_F_DENSITY,
                            &D_MBConv_1_0_PRJ_WEIGHTS, MBConv1_0_project_conv_conv2d_weights, 
                            MBCONV_1_0_PRJ_F_HEIGHT, MBCONV_1_0_PRJ_F_WIDTH, 
                            MBCONV_1_0_PRJ_F_DEPTH * MBCONV_1_0_PRJ_F_DENSITY); 

  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_1_EXPD_WEIGHTS, MBConv6_1_expansion_conv_conv2d_weights, 
                            MBCONV_6_1_EXPD_F_HEIGHT,   MBCONV_6_1_EXPD_F_WIDTH, 
                            MBCONV_6_1_EXPD_F_DEPTH * MBCONV_6_1_EXPD_F_DENSITY,
                            &D_MBConv_6_1_DW_WEIGHTS, MBConv6_1_depthwise_conv_conv2d_weights, 
                            MBCONV_6_1_DW_F_HEIGHT, MBCONV_6_1_DW_F_WIDTH, 
                            MBCONV_6_1_DW_F_DEPTH * MBCONV_6_1_DW_F_DENSITY,
                            &D_MBConv_6_1_SQZ_1_WEIGHTS, MBConv6_1_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_1_SQZ_1_F_HEIGHT, MBCONV_6_1_SQZ_1_F_WIDTH, 
                            MBCONV_6_1_SQZ_1_F_DEPTH * MBCONV_6_1_SQZ_1_F_DENSITY,
                            &D_MBConv_6_1_SQZ_2_WEIGHTS, MBConv6_1_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_1_SQZ_2_F_HEIGHT, MBCONV_6_1_SQZ_2_F_WIDTH, 
                            MBCONV_6_1_SQZ_2_F_DEPTH * MBCONV_6_1_SQZ_2_F_DENSITY,
                            &D_MBConv_6_1_PRJ_WEIGHTS, MBConv6_1_project_conv_conv2d_weights, 
                            MBCONV_6_1_PRJ_F_HEIGHT, MBCONV_6_1_PRJ_F_WIDTH, 
                            MBCONV_6_1_PRJ_F_DEPTH * MBCONV_6_1_PRJ_F_DENSITY); 

  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_2_EXPD_WEIGHTS, MBConv6_2_expansion_conv_conv2d_weights, 
                            MBCONV_6_2_EXPD_F_HEIGHT,   MBCONV_6_2_EXPD_F_WIDTH, 
                            MBCONV_6_2_EXPD_F_DEPTH * MBCONV_6_2_EXPD_F_DENSITY,
                            &D_MBConv_6_2_DW_WEIGHTS, MBConv6_2_depthwise_conv_conv2d_weights, 
                            MBCONV_6_2_DW_F_HEIGHT, MBCONV_6_2_DW_F_WIDTH, 
                            MBCONV_6_2_DW_F_DEPTH * MBCONV_6_2_DW_F_DENSITY,
                            &D_MBConv_6_2_SQZ_1_WEIGHTS, MBConv6_2_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_2_SQZ_1_F_HEIGHT, MBCONV_6_2_SQZ_1_F_WIDTH, 
                            MBCONV_6_2_SQZ_1_F_DEPTH * MBCONV_6_2_SQZ_1_F_DENSITY,
                            &D_MBConv_6_2_SQZ_2_WEIGHTS, MBConv6_2_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_2_SQZ_2_F_HEIGHT, MBCONV_6_2_SQZ_2_F_WIDTH, 
                            MBCONV_6_2_SQZ_2_F_DEPTH * MBCONV_6_2_SQZ_2_F_DENSITY,
                            &D_MBConv_6_2_PRJ_WEIGHTS, MBConv6_2_project_conv_conv2d_weights, 
                            MBCONV_6_2_PRJ_F_HEIGHT, MBCONV_6_2_PRJ_F_WIDTH, 
                            MBCONV_6_2_PRJ_F_DEPTH * MBCONV_6_2_PRJ_F_DENSITY);

  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_3_EXPD_WEIGHTS, MBConv6_3_expansion_conv_conv2d_weights, 
                            MBCONV_6_3_EXPD_F_HEIGHT,   MBCONV_6_3_EXPD_F_WIDTH, 
                            MBCONV_6_3_EXPD_F_DEPTH * MBCONV_6_3_EXPD_F_DENSITY,
                            &D_MBConv_6_3_DW_WEIGHTS, MBConv6_3_depthwise_conv_conv2d_weights, 
                            MBCONV_6_3_DW_F_HEIGHT, MBCONV_6_3_DW_F_WIDTH, 
                            MBCONV_6_3_DW_F_DEPTH * MBCONV_6_3_DW_F_DENSITY,
                            &D_MBConv_6_3_SQZ_1_WEIGHTS, MBConv6_3_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_3_SQZ_1_F_HEIGHT, MBCONV_6_3_SQZ_1_F_WIDTH, 
                            MBCONV_6_3_SQZ_1_F_DEPTH * MBCONV_6_3_SQZ_1_F_DENSITY,
                            &D_MBConv_6_3_SQZ_2_WEIGHTS, MBConv6_3_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_3_SQZ_2_F_HEIGHT, MBCONV_6_3_SQZ_2_F_WIDTH, 
                            MBCONV_6_3_SQZ_2_F_DEPTH * MBCONV_6_3_SQZ_2_F_DENSITY,
                            &D_MBConv_6_3_PRJ_WEIGHTS, MBConv6_3_project_conv_conv2d_weights, 
                            MBCONV_6_3_PRJ_F_HEIGHT, MBCONV_6_3_PRJ_F_WIDTH, 
                            MBCONV_6_3_PRJ_F_DEPTH * MBCONV_6_3_PRJ_F_DENSITY);


  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_4_EXPD_WEIGHTS, MBConv6_4_expansion_conv_conv2d_weights, 
                            MBCONV_6_4_EXPD_F_HEIGHT,   MBCONV_6_4_EXPD_F_WIDTH, 
                            MBCONV_6_4_EXPD_F_DEPTH * MBCONV_6_4_EXPD_F_DENSITY,
                            &D_MBConv_6_4_DW_WEIGHTS, MBConv6_4_depthwise_conv_conv2d_weights, 
                            MBCONV_6_4_DW_F_HEIGHT, MBCONV_6_4_DW_F_WIDTH, 
                            MBCONV_6_4_DW_F_DEPTH * MBCONV_6_4_DW_F_DENSITY,
                            &D_MBConv_6_4_SQZ_1_WEIGHTS, MBConv6_4_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_4_SQZ_1_F_HEIGHT, MBCONV_6_4_SQZ_1_F_WIDTH, 
                            MBCONV_6_4_SQZ_1_F_DEPTH * MBCONV_6_4_SQZ_1_F_DENSITY,
                            &D_MBConv_6_4_SQZ_2_WEIGHTS, MBConv6_4_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_4_SQZ_2_F_HEIGHT, MBCONV_6_4_SQZ_2_F_WIDTH, 
                            MBCONV_6_4_SQZ_2_F_DEPTH * MBCONV_6_4_SQZ_2_F_DENSITY,
                            &D_MBConv_6_4_PRJ_WEIGHTS, MBConv6_4_project_conv_conv2d_weights, 
                            MBCONV_6_4_PRJ_F_HEIGHT, MBCONV_6_4_PRJ_F_WIDTH, 
                            MBCONV_6_4_PRJ_F_DEPTH * MBCONV_6_4_PRJ_F_DENSITY);

  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_5_EXPD_WEIGHTS, MBConv6_5_expansion_conv_conv2d_weights, 
                            MBCONV_6_5_EXPD_F_HEIGHT,   MBCONV_6_5_EXPD_F_WIDTH, 
                            MBCONV_6_5_EXPD_F_DEPTH * MBCONV_6_5_EXPD_F_DENSITY,
                            &D_MBConv_6_5_DW_WEIGHTS, MBConv6_5_depthwise_conv_conv2d_weights, 
                            MBCONV_6_5_DW_F_HEIGHT, MBCONV_6_5_DW_F_WIDTH, 
                            MBCONV_6_5_DW_F_DEPTH * MBCONV_6_5_DW_F_DENSITY,
                            &D_MBConv_6_5_SQZ_1_WEIGHTS, MBConv6_5_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_5_SQZ_1_F_HEIGHT, MBCONV_6_5_SQZ_1_F_WIDTH, 
                            MBCONV_6_5_SQZ_1_F_DEPTH * MBCONV_6_5_SQZ_1_F_DENSITY,
                            &D_MBConv_6_5_SQZ_2_WEIGHTS, MBConv6_5_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_5_SQZ_2_F_HEIGHT, MBCONV_6_5_SQZ_2_F_WIDTH, 
                            MBCONV_6_5_SQZ_2_F_DEPTH * MBCONV_6_5_SQZ_2_F_DENSITY,
                            &D_MBConv_6_5_PRJ_WEIGHTS, MBConv6_5_project_conv_conv2d_weights, 
                            MBCONV_6_5_PRJ_F_HEIGHT, MBCONV_6_5_PRJ_F_WIDTH, 
                            MBCONV_6_5_PRJ_F_DEPTH * MBCONV_6_5_PRJ_F_DENSITY);
     
  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_6_EXPD_WEIGHTS, MBConv6_6_expansion_conv_conv2d_weights, 
                            MBCONV_6_6_EXPD_F_HEIGHT,   MBCONV_6_6_EXPD_F_WIDTH, 
                            MBCONV_6_6_EXPD_F_DEPTH * MBCONV_6_6_EXPD_F_DENSITY,
                            &D_MBConv_6_6_DW_WEIGHTS, MBConv6_6_depthwise_conv_conv2d_weights, 
                            MBCONV_6_6_DW_F_HEIGHT, MBCONV_6_6_DW_F_WIDTH, 
                            MBCONV_6_6_DW_F_DEPTH * MBCONV_6_6_DW_F_DENSITY,
                            &D_MBConv_6_6_SQZ_1_WEIGHTS, MBConv6_6_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_6_SQZ_1_F_HEIGHT, MBCONV_6_6_SQZ_1_F_WIDTH, 
                            MBCONV_6_6_SQZ_1_F_DEPTH * MBCONV_6_6_SQZ_1_F_DENSITY,
                            &D_MBConv_6_6_SQZ_2_WEIGHTS, MBConv6_6_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_6_SQZ_2_F_HEIGHT, MBCONV_6_6_SQZ_2_F_WIDTH, 
                            MBCONV_6_6_SQZ_2_F_DEPTH * MBCONV_6_6_SQZ_2_F_DENSITY,
                            &D_MBConv_6_6_PRJ_WEIGHTS, MBConv6_6_project_conv_conv2d_weights, 
                            MBCONV_6_6_PRJ_F_HEIGHT, MBCONV_6_6_PRJ_F_WIDTH, 
                            MBCONV_6_6_PRJ_F_DEPTH * MBCONV_6_6_PRJ_F_DENSITY);
     
  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_7_EXPD_WEIGHTS, MBConv6_7_expansion_conv_conv2d_weights, 
                            MBCONV_6_7_EXPD_F_HEIGHT,   MBCONV_6_7_EXPD_F_WIDTH, 
                            MBCONV_6_7_EXPD_F_DEPTH * MBCONV_6_7_EXPD_F_DENSITY,
                            &D_MBConv_6_7_DW_WEIGHTS, MBConv6_7_depthwise_conv_conv2d_weights, 
                            MBCONV_6_7_DW_F_HEIGHT, MBCONV_6_7_DW_F_WIDTH, 
                            MBCONV_6_7_DW_F_DEPTH * MBCONV_6_7_DW_F_DENSITY,
                            &D_MBConv_6_7_SQZ_1_WEIGHTS, MBConv6_7_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_7_SQZ_1_F_HEIGHT, MBCONV_6_7_SQZ_1_F_WIDTH, 
                            MBCONV_6_7_SQZ_1_F_DEPTH * MBCONV_6_7_SQZ_1_F_DENSITY,
                            &D_MBConv_6_7_SQZ_2_WEIGHTS, MBConv6_7_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_7_SQZ_2_F_HEIGHT, MBCONV_6_7_SQZ_2_F_WIDTH, 
                            MBCONV_6_7_SQZ_2_F_DEPTH * MBCONV_6_7_SQZ_2_F_DENSITY,
                            &D_MBConv_6_7_PRJ_WEIGHTS, MBConv6_7_project_conv_conv2d_weights, 
                            MBCONV_6_7_PRJ_F_HEIGHT, MBCONV_6_7_PRJ_F_WIDTH, 
                            MBCONV_6_7_PRJ_F_DEPTH * MBCONV_6_7_PRJ_F_DENSITY);


  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_8_EXPD_WEIGHTS, MBConv6_8_expansion_conv_conv2d_weights, 
                            MBCONV_6_8_EXPD_F_HEIGHT,   MBCONV_6_8_EXPD_F_WIDTH, 
                            MBCONV_6_8_EXPD_F_DEPTH * MBCONV_6_8_EXPD_F_DENSITY,
                            &D_MBConv_6_8_DW_WEIGHTS, MBConv6_8_depthwise_conv_conv2d_weights, 
                            MBCONV_6_8_DW_F_HEIGHT, MBCONV_6_8_DW_F_WIDTH, 
                            MBCONV_6_8_DW_F_DEPTH * MBCONV_6_8_DW_F_DENSITY,
                            &D_MBConv_6_8_SQZ_1_WEIGHTS, MBConv6_8_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_8_SQZ_1_F_HEIGHT, MBCONV_6_8_SQZ_1_F_WIDTH, 
                            MBCONV_6_8_SQZ_1_F_DEPTH * MBCONV_6_8_SQZ_1_F_DENSITY,
                            &D_MBConv_6_8_SQZ_2_WEIGHTS, MBConv6_8_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_8_SQZ_2_F_HEIGHT, MBCONV_6_8_SQZ_2_F_WIDTH, 
                            MBCONV_6_8_SQZ_2_F_DEPTH * MBCONV_6_8_SQZ_2_F_DENSITY,
                            &D_MBConv_6_8_PRJ_WEIGHTS, MBConv6_8_project_conv_conv2d_weights, 
                            MBCONV_6_8_PRJ_F_HEIGHT, MBCONV_6_8_PRJ_F_WIDTH, 
                            MBCONV_6_8_PRJ_F_DEPTH * MBCONV_6_8_PRJ_F_DENSITY);

  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_9_EXPD_WEIGHTS, MBConv6_9_expansion_conv_conv2d_weights, 
                            MBCONV_6_9_EXPD_F_HEIGHT,   MBCONV_6_9_EXPD_F_WIDTH, 
                            MBCONV_6_9_EXPD_F_DEPTH * MBCONV_6_9_EXPD_F_DENSITY,
                            &D_MBConv_6_9_DW_WEIGHTS, MBConv6_9_depthwise_conv_conv2d_weights, 
                            MBCONV_6_9_DW_F_HEIGHT, MBCONV_6_9_DW_F_WIDTH, 
                            MBCONV_6_9_DW_F_DEPTH * MBCONV_6_9_DW_F_DENSITY,
                            &D_MBConv_6_9_SQZ_1_WEIGHTS, MBConv6_9_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_9_SQZ_1_F_HEIGHT, MBCONV_6_9_SQZ_1_F_WIDTH, 
                            MBCONV_6_9_SQZ_1_F_DEPTH * MBCONV_6_9_SQZ_1_F_DENSITY,
                            &D_MBConv_6_9_SQZ_2_WEIGHTS, MBConv6_9_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_9_SQZ_2_F_HEIGHT, MBCONV_6_9_SQZ_2_F_WIDTH, 
                            MBCONV_6_9_SQZ_2_F_DEPTH * MBCONV_6_9_SQZ_2_F_DENSITY,
                            &D_MBConv_6_9_PRJ_WEIGHTS, MBConv6_9_project_conv_conv2d_weights, 
                            MBCONV_6_9_PRJ_F_HEIGHT, MBCONV_6_9_PRJ_F_WIDTH, 
                            MBCONV_6_9_PRJ_F_DEPTH * MBCONV_6_9_PRJ_F_DENSITY);


  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_10_EXPD_WEIGHTS, MBConv6_10_expansion_conv_conv2d_weights, 
                            MBCONV_6_10_EXPD_F_HEIGHT,   MBCONV_6_10_EXPD_F_WIDTH, 
                            MBCONV_6_10_EXPD_F_DEPTH * MBCONV_6_10_EXPD_F_DENSITY,
                            &D_MBConv_6_10_DW_WEIGHTS, MBConv6_10_depthwise_conv_conv2d_weights, 
                            MBCONV_6_10_DW_F_HEIGHT, MBCONV_6_10_DW_F_WIDTH, 
                            MBCONV_6_10_DW_F_DEPTH * MBCONV_6_10_DW_F_DENSITY,
                            &D_MBConv_6_10_SQZ_1_WEIGHTS, MBConv6_10_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_10_SQZ_1_F_HEIGHT, MBCONV_6_10_SQZ_1_F_WIDTH, 
                            MBCONV_6_10_SQZ_1_F_DEPTH * MBCONV_6_10_SQZ_1_F_DENSITY,
                            &D_MBConv_6_10_SQZ_2_WEIGHTS, MBConv6_10_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_10_SQZ_2_F_HEIGHT, MBCONV_6_10_SQZ_2_F_WIDTH, 
                            MBCONV_6_10_SQZ_2_F_DEPTH * MBCONV_6_10_SQZ_2_F_DENSITY,
                            &D_MBConv_6_10_PRJ_WEIGHTS, MBConv6_10_project_conv_conv2d_weights, 
                            MBCONV_6_10_PRJ_F_HEIGHT, MBCONV_6_10_PRJ_F_WIDTH, 
                            MBCONV_6_10_PRJ_F_DEPTH * MBCONV_6_10_PRJ_F_DENSITY);


  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_11_EXPD_WEIGHTS, MBConv6_11_expansion_conv_conv2d_weights, 
                            MBCONV_6_11_EXPD_F_HEIGHT,   MBCONV_6_11_EXPD_F_WIDTH, 
                            MBCONV_6_11_EXPD_F_DEPTH * MBCONV_6_11_EXPD_F_DENSITY,
                            &D_MBConv_6_11_DW_WEIGHTS, MBConv6_11_depthwise_conv_conv2d_weights, 
                            MBCONV_6_11_DW_F_HEIGHT, MBCONV_6_11_DW_F_WIDTH, 
                            MBCONV_6_11_DW_F_DEPTH * MBCONV_6_11_DW_F_DENSITY,
                            &D_MBConv_6_11_SQZ_1_WEIGHTS, MBConv6_11_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_11_SQZ_1_F_HEIGHT, MBCONV_6_11_SQZ_1_F_WIDTH, 
                            MBCONV_6_11_SQZ_1_F_DEPTH * MBCONV_6_11_SQZ_1_F_DENSITY,
                            &D_MBConv_6_11_SQZ_2_WEIGHTS, MBConv6_11_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_11_SQZ_2_F_HEIGHT, MBCONV_6_11_SQZ_2_F_WIDTH, 
                            MBCONV_6_11_SQZ_2_F_DEPTH * MBCONV_6_11_SQZ_2_F_DENSITY,
                            &D_MBConv_6_11_PRJ_WEIGHTS, MBConv6_11_project_conv_conv2d_weights, 
                            MBCONV_6_11_PRJ_F_HEIGHT, MBCONV_6_11_PRJ_F_WIDTH, 
                            MBCONV_6_11_PRJ_F_DEPTH * MBCONV_6_11_PRJ_F_DENSITY);

 
  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_12_EXPD_WEIGHTS, MBConv6_12_expansion_conv_conv2d_weights, 
                            MBCONV_6_12_EXPD_F_HEIGHT,   MBCONV_6_12_EXPD_F_WIDTH, 
                            MBCONV_6_12_EXPD_F_DEPTH * MBCONV_6_12_EXPD_F_DENSITY,
                            &D_MBConv_6_12_DW_WEIGHTS, MBConv6_12_depthwise_conv_conv2d_weights, 
                            MBCONV_6_12_DW_F_HEIGHT, MBCONV_6_12_DW_F_WIDTH, 
                            MBCONV_6_12_DW_F_DEPTH * MBCONV_6_12_DW_F_DENSITY,
                            &D_MBConv_6_12_SQZ_1_WEIGHTS, MBConv6_12_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_12_SQZ_1_F_HEIGHT, MBCONV_6_12_SQZ_1_F_WIDTH, 
                            MBCONV_6_12_SQZ_1_F_DEPTH * MBCONV_6_12_SQZ_1_F_DENSITY,
                            &D_MBConv_6_12_SQZ_2_WEIGHTS, MBConv6_12_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_12_SQZ_2_F_HEIGHT, MBCONV_6_12_SQZ_2_F_WIDTH, 
                            MBCONV_6_12_SQZ_2_F_DEPTH * MBCONV_6_12_SQZ_2_F_DENSITY,
                            &D_MBConv_6_12_PRJ_WEIGHTS, MBConv6_12_project_conv_conv2d_weights, 
                            MBCONV_6_12_PRJ_F_HEIGHT, MBCONV_6_12_PRJ_F_WIDTH, 
                            MBCONV_6_12_PRJ_F_DEPTH * MBCONV_6_12_PRJ_F_DENSITY);


  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_13_EXPD_WEIGHTS, MBConv6_13_expansion_conv_conv2d_weights, 
                            MBCONV_6_13_EXPD_F_HEIGHT,   MBCONV_6_13_EXPD_F_WIDTH, 
                            MBCONV_6_13_EXPD_F_DEPTH * MBCONV_6_13_EXPD_F_DENSITY,
                            &D_MBConv_6_13_DW_WEIGHTS, MBConv6_13_depthwise_conv_conv2d_weights, 
                            MBCONV_6_13_DW_F_HEIGHT, MBCONV_6_13_DW_F_WIDTH, 
                            MBCONV_6_13_DW_F_DEPTH * MBCONV_6_13_DW_F_DENSITY,
                            &D_MBConv_6_13_SQZ_1_WEIGHTS, MBConv6_13_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_13_SQZ_1_F_HEIGHT, MBCONV_6_13_SQZ_1_F_WIDTH, 
                            MBCONV_6_13_SQZ_1_F_DEPTH * MBCONV_6_13_SQZ_1_F_DENSITY,
                            &D_MBConv_6_13_SQZ_2_WEIGHTS, MBConv6_13_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_13_SQZ_2_F_HEIGHT, MBCONV_6_13_SQZ_2_F_WIDTH, 
                            MBCONV_6_13_SQZ_2_F_DEPTH * MBCONV_6_13_SQZ_2_F_DENSITY,
                            &D_MBConv_6_13_PRJ_WEIGHTS, MBConv6_13_project_conv_conv2d_weights, 
                            MBCONV_6_13_PRJ_F_HEIGHT, MBCONV_6_13_PRJ_F_WIDTH, 
                            MBCONV_6_13_PRJ_F_DEPTH * MBCONV_6_13_PRJ_F_DENSITY);


  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_14_EXPD_WEIGHTS, MBConv6_14_expansion_conv_conv2d_weights, 
                            MBCONV_6_14_EXPD_F_HEIGHT,   MBCONV_6_14_EXPD_F_WIDTH, 
                            MBCONV_6_14_EXPD_F_DEPTH * MBCONV_6_14_EXPD_F_DENSITY,
                            &D_MBConv_6_14_DW_WEIGHTS, MBConv6_14_depthwise_conv_conv2d_weights, 
                            MBCONV_6_14_DW_F_HEIGHT, MBCONV_6_14_DW_F_WIDTH, 
                            MBCONV_6_14_DW_F_DEPTH * MBCONV_6_14_DW_F_DENSITY,
                            &D_MBConv_6_14_SQZ_1_WEIGHTS, MBConv6_14_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_14_SQZ_1_F_HEIGHT, MBCONV_6_14_SQZ_1_F_WIDTH, 
                            MBCONV_6_14_SQZ_1_F_DEPTH * MBCONV_6_14_SQZ_1_F_DENSITY,
                            &D_MBConv_6_14_SQZ_2_WEIGHTS, MBConv6_14_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_14_SQZ_2_F_HEIGHT, MBCONV_6_14_SQZ_2_F_WIDTH, 
                            MBCONV_6_14_SQZ_2_F_DEPTH * MBCONV_6_14_SQZ_2_F_DENSITY,
                            &D_MBConv_6_14_PRJ_WEIGHTS, MBConv6_14_project_conv_conv2d_weights, 
                            MBCONV_6_14_PRJ_F_HEIGHT, MBCONV_6_14_PRJ_F_WIDTH, 
                            MBCONV_6_14_PRJ_F_DEPTH * MBCONV_6_14_PRJ_F_DENSITY);
     
  DEFINE_FILTERS_FOR_MBCONV(&D_MBConv_6_15_EXPD_WEIGHTS, MBConv6_15_expansion_conv_conv2d_weights, 
                            MBCONV_6_15_EXPD_F_HEIGHT,   MBCONV_6_15_EXPD_F_WIDTH, 
                            MBCONV_6_15_EXPD_F_DEPTH * MBCONV_6_15_EXPD_F_DENSITY,
                            &D_MBConv_6_15_DW_WEIGHTS, MBConv6_15_depthwise_conv_conv2d_weights, 
                            MBCONV_6_15_DW_F_HEIGHT, MBCONV_6_15_DW_F_WIDTH, 
                            MBCONV_6_15_DW_F_DEPTH * MBCONV_6_15_DW_F_DENSITY,
                            &D_MBConv_6_15_SQZ_1_WEIGHTS, MBConv6_15_squeeze_excitation1_conv2d_weights,
                            MBCONV_6_15_SQZ_1_F_HEIGHT, MBCONV_6_15_SQZ_1_F_WIDTH, 
                            MBCONV_6_15_SQZ_1_F_DEPTH * MBCONV_6_15_SQZ_1_F_DENSITY,
                            &D_MBConv_6_15_SQZ_2_WEIGHTS, MBConv6_15_squeeze_excitation2_conv2d_weights, 
                            MBCONV_6_15_SQZ_2_F_HEIGHT, MBCONV_6_15_SQZ_2_F_WIDTH, 
                            MBCONV_6_15_SQZ_2_F_DEPTH * MBCONV_6_15_SQZ_2_F_DENSITY,
                            &D_MBConv_6_15_PRJ_WEIGHTS, MBConv6_15_project_conv_conv2d_weights, 
                            MBCONV_6_15_PRJ_F_HEIGHT, MBCONV_6_15_PRJ_F_WIDTH, 
                            MBCONV_6_15_PRJ_F_DEPTH * MBCONV_6_15_PRJ_F_DENSITY);


  set_allocate_copy_array_Device(&HEAD_CONV_WEIGHTS, Head_conv2d_weights,
                                  HEAD_CONV_F_HEIGHT, HEAD_CONV_F_WIDTH, HEAD_CONV_F_DEPTH * HEAD_CONV_F_DENSITY,
                                "Head Filter  is allocated in device memory");   
 
  set_allocate_copy_array_Device(&HEAD_FC_WEIGHTS, Head_linear_weights,
                                HEAD_FC_F_HEIGHT, HEAD_FC_F_WIDTH, 1,
                                "Fully Connected weights matrix is allocated in device memory");  
  
  // Define bias matrices for all squeeze layers
  set_allocate_copy_array_Device(&MBConv6_15_SQZ_1_bias, MBConv6_15_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_15_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #15");  
  set_allocate_copy_array_Device(&MBConv6_14_SQZ_1_bias, MBConv6_14_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_14_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #14");
  set_allocate_copy_array_Device(&MBConv6_13_SQZ_1_bias, MBConv6_13_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_13_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #13");
  set_allocate_copy_array_Device(&MBConv6_12_SQZ_1_bias, MBConv6_12_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_12_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #12");
  set_allocate_copy_array_Device(&MBConv6_11_SQZ_1_bias, MBConv6_11_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_11_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #11");
  set_allocate_copy_array_Device(&MBConv6_10_SQZ_1_bias, MBConv6_10_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_10_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #10");  
  set_allocate_copy_array_Device(&MBConv6_9_SQZ_1_bias, MBConv6_9_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_9_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #9");
  set_allocate_copy_array_Device(&MBConv6_8_SQZ_1_bias, MBConv6_8_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_8_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #8");
  set_allocate_copy_array_Device(&MBConv6_7_SQZ_1_bias, MBConv6_7_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_7_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #7");
  set_allocate_copy_array_Device(&MBConv6_6_SQZ_1_bias, MBConv6_6_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_6_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #6");
  set_allocate_copy_array_Device(&MBConv6_5_SQZ_1_bias, MBConv6_5_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_5_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #5");  
  set_allocate_copy_array_Device(&MBConv6_4_SQZ_1_bias, MBConv6_4_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_4_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #4");
  set_allocate_copy_array_Device(&MBConv6_3_SQZ_1_bias, MBConv6_3_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_3_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #3");
  set_allocate_copy_array_Device(&MBConv6_2_SQZ_1_bias, MBConv6_2_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_2_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #2");
  set_allocate_copy_array_Device(&MBConv6_1_SQZ_1_bias, MBConv6_1_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv6_1_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #1");
  set_allocate_copy_array_Device(&MBConv1_0_SQZ_1_bias, MBConv1_0_squeeze_excitation1_conv2d_bias,
                                  sizeof(MBConv1_0_squeeze_excitation1_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 1 layer #0");   
  set_allocate_copy_array_Device(&MBConv6_15_SQZ_2_bias, MBConv6_15_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_15_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #15");  
  set_allocate_copy_array_Device(&MBConv6_14_SQZ_2_bias, MBConv6_14_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_14_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #14");
  set_allocate_copy_array_Device(&MBConv6_13_SQZ_2_bias, MBConv6_13_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_13_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #13");
  set_allocate_copy_array_Device(&MBConv6_12_SQZ_2_bias, MBConv6_12_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_12_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #12");
  set_allocate_copy_array_Device(&MBConv6_11_SQZ_2_bias, MBConv6_11_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_11_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #11");
  set_allocate_copy_array_Device(&MBConv6_10_SQZ_2_bias, MBConv6_10_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_10_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #10");  
  set_allocate_copy_array_Device(&MBConv6_9_SQZ_2_bias, MBConv6_9_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_9_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #9");
  set_allocate_copy_array_Device(&MBConv6_8_SQZ_2_bias, MBConv6_8_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_8_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #8");
  set_allocate_copy_array_Device(&MBConv6_7_SQZ_2_bias, MBConv6_7_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_7_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #7");
  set_allocate_copy_array_Device(&MBConv6_6_SQZ_2_bias, MBConv6_6_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_6_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #6");
  set_allocate_copy_array_Device(&MBConv6_5_SQZ_2_bias, MBConv6_5_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_5_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #5");  
  set_allocate_copy_array_Device(&MBConv6_4_SQZ_2_bias, MBConv6_4_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_4_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #4");
  set_allocate_copy_array_Device(&MBConv6_3_SQZ_2_bias, MBConv6_3_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_3_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #3");
  set_allocate_copy_array_Device(&MBConv6_2_SQZ_2_bias, MBConv6_2_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_2_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #2");
  set_allocate_copy_array_Device(&MBConv6_1_SQZ_2_bias, MBConv6_1_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv6_1_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #1");
  set_allocate_copy_array_Device(&MBConv1_0_SQZ_2_bias, MBConv1_0_squeeze_excitation2_conv2d_bias,
                                  sizeof(MBConv1_0_squeeze_excitation2_conv2d_bias)/sizeof(float), 1, 1,
                                  "Bias for squeeze 2 layer #0");    
 
// 3. Define BN mean,variance, weights and bias
MBCONV1_0_flag = 1;
 
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv1_0_EXPD_BN_MEAN,      NULL, 0,
								&MBConv1_0_EXPD_BN_VARIANCE,	NULL, 0,
								&MBConv1_0_EXPD_BN_WEIGHTS,		NULL, 0,
								&MBConv1_0_EXPD_BN_BIAS,			NULL, 0,

								&MBConv1_0_DW_BN_MEAN,        MBConv1_0_depthwise_conv_BN_mean,		  sizeof(MBConv1_0_depthwise_conv_BN_mean) / sizeof(float), 		
								&MBConv1_0_DW_BN_VARIANCE,		MBConv1_0_depthwise_conv_BN_variance,	sizeof(MBConv1_0_depthwise_conv_BN_variance) / sizeof(float),
								&MBConv1_0_DW_BN_WEIGHTS,     MBConv1_0_depthwise_conv_BN_weights,	sizeof(MBConv1_0_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv1_0_DW_BN_BIAS,				MBConv1_0_depthwise_conv_BN_bias,		  sizeof(MBConv1_0_depthwise_conv_BN_bias) / sizeof(float),

								&MBConv1_0_PRJ_BN_MEAN,       MBConv1_0_project_conv_BN_mean,			  sizeof(MBConv1_0_project_conv_BN_mean) / sizeof(float),
								&MBConv1_0_PRJ_BN_VARIANCE,		MBConv1_0_project_conv_BN_variance,		sizeof(MBConv1_0_project_conv_BN_variance) / sizeof(float),
								&MBConv1_0_PRJ_BN_WEIGHTS,    MBConv1_0_project_conv_BN_weights,		sizeof(MBConv1_0_project_conv_BN_weights) / sizeof(float),
								&MBConv1_0_PRJ_BN_BIAS,				MBConv1_0_project_conv_BN_bias, 		  sizeof(MBConv1_0_project_conv_BN_bias) / sizeof(float));

MBCONV1_0_flag = 0;
 
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_1_EXPD_BN_MEAN,      MBConv6_1_expansion_conv_BN_mean,		  sizeof(MBConv6_1_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_1_EXPD_BN_VARIANCE,	MBConv6_1_expansion_conv_BN_variance,	sizeof(MBConv6_1_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_1_EXPD_BN_WEIGHTS,   MBConv6_1_expansion_conv_BN_weights,	sizeof(MBConv6_1_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_1_EXPD_BN_BIAS,			MBConv6_1_expansion_conv_BN_bias,		  sizeof(MBConv6_1_expansion_conv_BN_bias) / sizeof(float),
								
								&MBConv6_1_DW_BN_MEAN,        MBConv6_1_depthwise_conv_BN_mean,		  sizeof(MBConv6_1_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_1_DW_BN_VARIANCE,		MBConv6_1_depthwise_conv_BN_variance,	sizeof(MBConv6_1_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_1_DW_BN_WEIGHTS,     MBConv6_1_depthwise_conv_BN_weights,	sizeof(MBConv6_1_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_1_DW_BN_BIAS,				MBConv6_1_depthwise_conv_BN_bias,		  sizeof(MBConv6_1_depthwise_conv_BN_bias) / sizeof(float),
								
								&MBConv6_1_PRJ_BN_MEAN,       MBConv6_1_project_conv_BN_mean,			  sizeof(MBConv6_1_project_conv_BN_mean) / sizeof(float),
								&MBConv6_1_PRJ_BN_VARIANCE,		MBConv6_1_project_conv_BN_variance,		sizeof(MBConv6_1_project_conv_BN_variance) / sizeof(float),
								&MBConv6_1_PRJ_BN_WEIGHTS,    MBConv6_1_project_conv_BN_weights,		sizeof(MBConv6_1_project_conv_BN_weights) / sizeof(float),
								&MBConv6_1_PRJ_BN_BIAS,				MBConv6_1_project_conv_BN_bias, 		  sizeof(MBConv6_1_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_2_EXPD_BN_MEAN,      MBConv6_2_expansion_conv_BN_mean,		  sizeof(MBConv6_2_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_2_EXPD_BN_VARIANCE,	MBConv6_2_expansion_conv_BN_variance,	sizeof(MBConv6_2_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_2_EXPD_BN_WEIGHTS,   MBConv6_2_expansion_conv_BN_weights,	sizeof(MBConv6_2_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_2_EXPD_BN_BIAS,			MBConv6_2_expansion_conv_BN_bias,		  sizeof(MBConv6_2_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_2_DW_BN_MEAN,        MBConv6_2_depthwise_conv_BN_mean,		  sizeof(MBConv6_2_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_2_DW_BN_VARIANCE,		MBConv6_2_depthwise_conv_BN_variance,	sizeof(MBConv6_2_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_2_DW_BN_WEIGHTS,     MBConv6_2_depthwise_conv_BN_weights,	sizeof(MBConv6_2_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_2_DW_BN_BIAS,			  MBConv6_2_depthwise_conv_BN_bias,		  sizeof(MBConv6_2_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_2_PRJ_BN_MEAN,       MBConv6_2_project_conv_BN_mean,			  sizeof(MBConv6_2_project_conv_BN_mean) / sizeof(float),
								&MBConv6_2_PRJ_BN_VARIANCE,		MBConv6_2_project_conv_BN_variance,		sizeof(MBConv6_2_project_conv_BN_variance) / sizeof(float),
								&MBConv6_2_PRJ_BN_WEIGHTS,    MBConv6_2_project_conv_BN_weights,		sizeof(MBConv6_2_project_conv_BN_weights) / sizeof(float),
								&MBConv6_2_PRJ_BN_BIAS,				MBConv6_2_project_conv_BN_bias, 		  sizeof(MBConv6_2_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_3_EXPD_BN_MEAN,      MBConv6_3_expansion_conv_BN_mean, 		sizeof(MBConv6_3_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_3_EXPD_BN_VARIANCE,	MBConv6_3_expansion_conv_BN_variance,	sizeof(MBConv6_3_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_3_EXPD_BN_WEIGHTS,   MBConv6_3_expansion_conv_BN_weights,	sizeof(MBConv6_3_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_3_EXPD_BN_BIAS,			MBConv6_3_expansion_conv_BN_bias,		  sizeof(MBConv6_3_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_3_DW_BN_MEAN,        MBConv6_3_depthwise_conv_BN_mean,		  sizeof(MBConv6_3_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_3_DW_BN_VARIANCE,		MBConv6_3_depthwise_conv_BN_variance,	sizeof(MBConv6_3_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_3_DW_BN_WEIGHTS,     MBConv6_3_depthwise_conv_BN_weights,	sizeof(MBConv6_3_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_3_DW_BN_BIAS,				MBConv6_3_depthwise_conv_BN_bias,		  sizeof(MBConv6_3_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_3_PRJ_BN_MEAN,       MBConv6_3_project_conv_BN_mean,			  sizeof(MBConv6_3_project_conv_BN_mean) / sizeof(float),
								&MBConv6_3_PRJ_BN_VARIANCE,		MBConv6_3_project_conv_BN_variance,		sizeof(MBConv6_3_project_conv_BN_variance) / sizeof(float),
								&MBConv6_3_PRJ_BN_WEIGHTS,    MBConv6_3_project_conv_BN_weights,		sizeof(MBConv6_3_project_conv_BN_weights) / sizeof(float),
								&MBConv6_3_PRJ_BN_BIAS,				MBConv6_3_project_conv_BN_bias, 		  sizeof(MBConv6_3_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_4_EXPD_BN_MEAN,      MBConv6_4_expansion_conv_BN_mean, 		sizeof(MBConv6_4_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_4_EXPD_BN_VARIANCE,	MBConv6_4_expansion_conv_BN_variance,	sizeof(MBConv6_4_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_4_EXPD_BN_WEIGHTS,   MBConv6_4_expansion_conv_BN_weights,	sizeof(MBConv6_4_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_4_EXPD_BN_BIAS,			MBConv6_4_expansion_conv_BN_bias,		  sizeof(MBConv6_4_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_4_DW_BN_MEAN,        MBConv6_4_depthwise_conv_BN_mean,		  sizeof(MBConv6_4_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_4_DW_BN_VARIANCE,		MBConv6_4_depthwise_conv_BN_variance,	sizeof(MBConv6_4_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_4_DW_BN_WEIGHTS,     MBConv6_4_depthwise_conv_BN_weights,	sizeof(MBConv6_4_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_4_DW_BN_BIAS,				MBConv6_4_depthwise_conv_BN_bias,		  sizeof(MBConv6_4_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_4_PRJ_BN_MEAN,       MBConv6_4_project_conv_BN_mean,			  sizeof(MBConv6_4_project_conv_BN_mean) / sizeof(float),
								&MBConv6_4_PRJ_BN_VARIANCE,		MBConv6_4_project_conv_BN_variance,		sizeof(MBConv6_4_project_conv_BN_variance) / sizeof(float),
								&MBConv6_4_PRJ_BN_WEIGHTS,    MBConv6_4_project_conv_BN_weights,		sizeof(MBConv6_4_project_conv_BN_weights) / sizeof(float),
                &MBConv6_4_PRJ_BN_BIAS,				MBConv6_4_project_conv_BN_bias, 		  sizeof(MBConv6_4_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_5_EXPD_BN_MEAN,      MBConv6_5_expansion_conv_BN_mean,		  sizeof(MBConv6_5_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_5_EXPD_BN_VARIANCE,	MBConv6_5_expansion_conv_BN_variance,	sizeof(MBConv6_5_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_5_EXPD_BN_WEIGHTS,   MBConv6_5_expansion_conv_BN_weights,	sizeof(MBConv6_5_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_5_EXPD_BN_BIAS,			MBConv6_5_expansion_conv_BN_bias,		  sizeof(MBConv6_5_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_5_DW_BN_MEAN,        MBConv6_5_depthwise_conv_BN_mean,		  sizeof(MBConv6_5_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_5_DW_BN_VARIANCE,		MBConv6_5_depthwise_conv_BN_variance,	sizeof(MBConv6_5_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_5_DW_BN_WEIGHTS,     MBConv6_5_depthwise_conv_BN_weights,	sizeof(MBConv6_5_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_5_DW_BN_BIAS,				MBConv6_5_depthwise_conv_BN_bias,		  sizeof(MBConv6_5_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_5_PRJ_BN_MEAN,       MBConv6_5_project_conv_BN_mean,			  sizeof(MBConv6_5_project_conv_BN_mean) / sizeof(float),
								&MBConv6_5_PRJ_BN_VARIANCE,		MBConv6_5_project_conv_BN_variance,		sizeof(MBConv6_5_project_conv_BN_variance) / sizeof(float),
								&MBConv6_5_PRJ_BN_WEIGHTS,    MBConv6_5_project_conv_BN_weights,		sizeof(MBConv6_5_project_conv_BN_weights) / sizeof(float),
								&MBConv6_5_PRJ_BN_BIAS,				MBConv6_5_project_conv_BN_bias, 		  sizeof(MBConv6_5_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_6_EXPD_BN_MEAN,      MBConv6_6_expansion_conv_BN_mean,		  sizeof(MBConv6_6_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_6_EXPD_BN_VARIANCE,	MBConv6_6_expansion_conv_BN_variance,	sizeof(MBConv6_6_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_6_EXPD_BN_WEIGHTS,   MBConv6_6_expansion_conv_BN_weights,	sizeof(MBConv6_6_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_6_EXPD_BN_BIAS,			MBConv6_6_expansion_conv_BN_bias,		  sizeof(MBConv6_6_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_6_DW_BN_MEAN,        MBConv6_6_depthwise_conv_BN_mean,		  sizeof(MBConv6_6_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_6_DW_BN_VARIANCE,		MBConv6_6_depthwise_conv_BN_variance,	sizeof(MBConv6_6_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_6_DW_BN_WEIGHTS,     MBConv6_6_depthwise_conv_BN_weights,	sizeof(MBConv6_6_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_6_DW_BN_BIAS,				MBConv6_6_depthwise_conv_BN_bias,		  sizeof(MBConv6_6_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_6_PRJ_BN_MEAN,       MBConv6_6_project_conv_BN_mean,			  sizeof(MBConv6_6_project_conv_BN_mean) / sizeof(float),
								&MBConv6_6_PRJ_BN_VARIANCE,		MBConv6_6_project_conv_BN_variance,		sizeof(MBConv6_6_project_conv_BN_variance) / sizeof(float),
								&MBConv6_6_PRJ_BN_WEIGHTS,    MBConv6_6_project_conv_BN_weights,		sizeof(MBConv6_6_project_conv_BN_weights) / sizeof(float),
								&MBConv6_6_PRJ_BN_BIAS,				MBConv6_6_project_conv_BN_bias, 		  sizeof(MBConv6_6_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_7_EXPD_BN_MEAN,      MBConv6_7_expansion_conv_BN_mean,		  sizeof(MBConv6_7_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_7_EXPD_BN_VARIANCE,	MBConv6_7_expansion_conv_BN_variance,	sizeof(MBConv6_7_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_7_EXPD_BN_WEIGHTS,   MBConv6_7_expansion_conv_BN_weights,	sizeof(MBConv6_7_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_7_EXPD_BN_BIAS,			MBConv6_7_expansion_conv_BN_bias,		  sizeof(MBConv6_7_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_7_DW_BN_MEAN,        MBConv6_7_depthwise_conv_BN_mean,		  sizeof(MBConv6_7_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_7_DW_BN_VARIANCE,		MBConv6_7_depthwise_conv_BN_variance,	sizeof(MBConv6_7_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_7_DW_BN_WEIGHTS,     MBConv6_7_depthwise_conv_BN_weights,	sizeof(MBConv6_7_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_7_DW_BN_BIAS,				MBConv6_7_depthwise_conv_BN_bias,		  sizeof(MBConv6_7_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_7_PRJ_BN_MEAN,       MBConv6_7_project_conv_BN_mean,			  sizeof(MBConv6_7_project_conv_BN_mean) / sizeof(float),
								&MBConv6_7_PRJ_BN_VARIANCE,		MBConv6_7_project_conv_BN_variance,		sizeof(MBConv6_7_project_conv_BN_variance) / sizeof(float),
								&MBConv6_7_PRJ_BN_WEIGHTS,    MBConv6_7_project_conv_BN_weights,		sizeof(MBConv6_7_project_conv_BN_weights) / sizeof(float),
								&MBConv6_7_PRJ_BN_BIAS,				MBConv6_7_project_conv_BN_bias, 		  sizeof(MBConv6_7_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_8_EXPD_BN_MEAN,      MBConv6_8_expansion_conv_BN_mean,		  sizeof(MBConv6_8_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_8_EXPD_BN_VARIANCE,	MBConv6_8_expansion_conv_BN_variance,	sizeof(MBConv6_8_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_8_EXPD_BN_WEIGHTS,   MBConv6_8_expansion_conv_BN_weights,	sizeof(MBConv6_8_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_8_EXPD_BN_BIAS,			MBConv6_8_expansion_conv_BN_bias,		  sizeof(MBConv6_8_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_8_DW_BN_MEAN,        MBConv6_8_depthwise_conv_BN_mean,		  sizeof(MBConv6_8_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_8_DW_BN_VARIANCE,		MBConv6_8_depthwise_conv_BN_variance,	sizeof(MBConv6_8_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_8_DW_BN_WEIGHTS,     MBConv6_8_depthwise_conv_BN_weights,	sizeof(MBConv6_8_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_8_DW_BN_BIAS,				MBConv6_8_depthwise_conv_BN_bias,		  sizeof(MBConv6_8_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_8_PRJ_BN_MEAN,       MBConv6_8_project_conv_BN_mean,			  sizeof(MBConv6_8_project_conv_BN_mean) / sizeof(float),
								&MBConv6_8_PRJ_BN_VARIANCE,		MBConv6_8_project_conv_BN_variance,		sizeof(MBConv6_8_project_conv_BN_variance) / sizeof(float),
								&MBConv6_8_PRJ_BN_WEIGHTS,    MBConv6_8_project_conv_BN_weights,		sizeof(MBConv6_8_project_conv_BN_weights) / sizeof(float),
								&MBConv6_8_PRJ_BN_BIAS,				MBConv6_8_project_conv_BN_bias, 		  sizeof(MBConv6_8_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_9_EXPD_BN_MEAN,      MBConv6_9_expansion_conv_BN_mean,		  sizeof(MBConv6_9_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_9_EXPD_BN_VARIANCE,	MBConv6_9_expansion_conv_BN_variance,	sizeof(MBConv6_9_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_9_EXPD_BN_WEIGHTS,   MBConv6_9_expansion_conv_BN_weights,	sizeof(MBConv6_9_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_9_EXPD_BN_BIAS,			MBConv6_9_expansion_conv_BN_bias,		  sizeof(MBConv6_9_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_9_DW_BN_MEAN,        MBConv6_9_depthwise_conv_BN_mean,		  sizeof(MBConv6_9_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_9_DW_BN_VARIANCE,		MBConv6_9_depthwise_conv_BN_variance,	sizeof(MBConv6_9_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_9_DW_BN_WEIGHTS,     MBConv6_9_depthwise_conv_BN_weights,	sizeof(MBConv6_9_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_9_DW_BN_BIAS,				MBConv6_9_depthwise_conv_BN_bias,		  sizeof(MBConv6_9_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_9_PRJ_BN_MEAN,       MBConv6_9_project_conv_BN_mean,			  sizeof(MBConv6_9_project_conv_BN_mean) / sizeof(float),
								&MBConv6_9_PRJ_BN_VARIANCE,		MBConv6_9_project_conv_BN_variance,		sizeof(MBConv6_9_project_conv_BN_variance) / sizeof(float),
								&MBConv6_9_PRJ_BN_WEIGHTS,    MBConv6_9_project_conv_BN_weights,		sizeof(MBConv6_9_project_conv_BN_weights) / sizeof(float),
								&MBConv6_9_PRJ_BN_BIAS,				MBConv6_9_project_conv_BN_bias, 		  sizeof(MBConv6_9_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_10_EXPD_BN_MEAN,     MBConv6_10_expansion_conv_BN_mean,    sizeof(MBConv6_10_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_10_EXPD_BN_VARIANCE,	MBConv6_10_expansion_conv_BN_variance,sizeof(MBConv6_10_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_10_EXPD_BN_WEIGHTS,  MBConv6_10_expansion_conv_BN_weights,	sizeof(MBConv6_10_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_10_EXPD_BN_BIAS,			MBConv6_10_expansion_conv_BN_bias,		sizeof(MBConv6_10_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_10_DW_BN_MEAN,       MBConv6_10_depthwise_conv_BN_mean,		sizeof(MBConv6_10_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_10_DW_BN_VARIANCE,		MBConv6_10_depthwise_conv_BN_variance,sizeof(MBConv6_10_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_10_DW_BN_WEIGHTS,    MBConv6_10_depthwise_conv_BN_weights,	sizeof(MBConv6_10_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_10_DW_BN_BIAS,				MBConv6_10_depthwise_conv_BN_bias,		sizeof(MBConv6_10_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_10_PRJ_BN_MEAN,      MBConv6_10_project_conv_BN_mean,		  sizeof(MBConv6_10_project_conv_BN_mean) / sizeof(float),
								&MBConv6_10_PRJ_BN_VARIANCE,	MBConv6_10_project_conv_BN_variance,	sizeof(MBConv6_10_project_conv_BN_variance) / sizeof(float),
								&MBConv6_10_PRJ_BN_WEIGHTS,   MBConv6_10_project_conv_BN_weights,		sizeof(MBConv6_10_project_conv_BN_weights) / sizeof(float),
								&MBConv6_10_PRJ_BN_BIAS,			MBConv6_10_project_conv_BN_bias, 		  sizeof(MBConv6_10_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_11_EXPD_BN_MEAN,     MBConv6_11_expansion_conv_BN_mean,		sizeof(MBConv6_11_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_11_EXPD_BN_VARIANCE,	MBConv6_11_expansion_conv_BN_variance,sizeof(MBConv6_11_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_11_EXPD_BN_WEIGHTS,  MBConv6_11_expansion_conv_BN_weights,	sizeof(MBConv6_11_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_11_EXPD_BN_BIAS,			MBConv6_11_expansion_conv_BN_bias,		sizeof(MBConv6_11_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_11_DW_BN_MEAN,       MBConv6_11_depthwise_conv_BN_mean,		sizeof(MBConv6_11_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_11_DW_BN_VARIANCE,		MBConv6_11_depthwise_conv_BN_variance,sizeof(MBConv6_11_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_11_DW_BN_WEIGHTS,    MBConv6_11_depthwise_conv_BN_weights,	sizeof(MBConv6_11_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_11_DW_BN_BIAS,				MBConv6_11_depthwise_conv_BN_bias,		sizeof(MBConv6_11_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_11_PRJ_BN_MEAN,      MBConv6_11_project_conv_BN_mean,		  sizeof(MBConv6_11_project_conv_BN_mean) / sizeof(float),
								&MBConv6_11_PRJ_BN_VARIANCE,	MBConv6_11_project_conv_BN_variance,	sizeof(MBConv6_11_project_conv_BN_variance) / sizeof(float),
								&MBConv6_11_PRJ_BN_WEIGHTS,   MBConv6_11_project_conv_BN_weights,		sizeof(MBConv6_11_project_conv_BN_weights) / sizeof(float),
								&MBConv6_11_PRJ_BN_BIAS,			MBConv6_11_project_conv_BN_bias, 		  sizeof(MBConv6_11_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_12_EXPD_BN_MEAN,     MBConv6_12_expansion_conv_BN_mean,		sizeof(MBConv6_12_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_12_EXPD_BN_VARIANCE,	MBConv6_12_expansion_conv_BN_variance,sizeof(MBConv6_12_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_12_EXPD_BN_WEIGHTS,  MBConv6_12_expansion_conv_BN_weights,	sizeof(MBConv6_12_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_12_EXPD_BN_BIAS,			MBConv6_12_expansion_conv_BN_bias,		sizeof(MBConv6_12_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_12_DW_BN_MEAN,       MBConv6_12_depthwise_conv_BN_mean,		sizeof(MBConv6_12_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_12_DW_BN_VARIANCE,		MBConv6_12_depthwise_conv_BN_variance,sizeof(MBConv6_12_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_12_DW_BN_WEIGHTS,    MBConv6_12_depthwise_conv_BN_weights,	sizeof(MBConv6_12_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_12_DW_BN_BIAS,				MBConv6_12_depthwise_conv_BN_bias,		sizeof(MBConv6_12_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_12_PRJ_BN_MEAN,      MBConv6_12_project_conv_BN_mean,		  sizeof(MBConv6_12_project_conv_BN_mean) / sizeof(float),
								&MBConv6_12_PRJ_BN_VARIANCE,	MBConv6_12_project_conv_BN_variance,	sizeof(MBConv6_12_project_conv_BN_variance) / sizeof(float),
								&MBConv6_12_PRJ_BN_WEIGHTS,   MBConv6_12_project_conv_BN_weights,		sizeof(MBConv6_12_project_conv_BN_weights) / sizeof(float),
								&MBConv6_12_PRJ_BN_BIAS,			MBConv6_12_project_conv_BN_bias, 		  sizeof(MBConv6_12_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_13_EXPD_BN_MEAN,     MBConv6_13_expansion_conv_BN_mean,		sizeof(MBConv6_13_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_13_EXPD_BN_VARIANCE,	MBConv6_13_expansion_conv_BN_variance,sizeof(MBConv6_13_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_13_EXPD_BN_WEIGHTS,  MBConv6_13_expansion_conv_BN_weights,	sizeof(MBConv6_13_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_13_EXPD_BN_BIAS,			MBConv6_13_expansion_conv_BN_bias,		sizeof(MBConv6_13_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_13_DW_BN_MEAN,       MBConv6_13_depthwise_conv_BN_mean,		sizeof(MBConv6_13_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_13_DW_BN_VARIANCE,		MBConv6_13_depthwise_conv_BN_variance,sizeof(MBConv6_13_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_13_DW_BN_WEIGHTS,    MBConv6_13_depthwise_conv_BN_weights,	sizeof(MBConv6_13_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_13_DW_BN_BIAS,				MBConv6_13_depthwise_conv_BN_bias,		sizeof(MBConv6_13_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_13_PRJ_BN_MEAN,      MBConv6_13_project_conv_BN_mean,		  sizeof(MBConv6_13_project_conv_BN_mean) / sizeof(float),
								&MBConv6_13_PRJ_BN_VARIANCE,	MBConv6_13_project_conv_BN_variance,	sizeof(MBConv6_13_project_conv_BN_variance) / sizeof(float),
								&MBConv6_13_PRJ_BN_WEIGHTS,   MBConv6_13_project_conv_BN_weights,		sizeof(MBConv6_13_project_conv_BN_weights) / sizeof(float),
								&MBConv6_13_PRJ_BN_BIAS,			MBConv6_13_project_conv_BN_bias, 		  sizeof(MBConv6_13_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_14_EXPD_BN_MEAN,     MBConv6_14_expansion_conv_BN_mean,		sizeof(MBConv6_14_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_14_EXPD_BN_VARIANCE,	MBConv6_14_expansion_conv_BN_variance,sizeof(MBConv6_14_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_14_EXPD_BN_WEIGHTS,  MBConv6_14_expansion_conv_BN_weights,	sizeof(MBConv6_14_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_14_EXPD_BN_BIAS,			MBConv6_14_expansion_conv_BN_bias,		sizeof(MBConv6_14_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_14_DW_BN_MEAN,       MBConv6_14_depthwise_conv_BN_mean,		sizeof(MBConv6_14_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_14_DW_BN_VARIANCE,		MBConv6_14_depthwise_conv_BN_variance,sizeof(MBConv6_14_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_14_DW_BN_WEIGHTS,    MBConv6_14_depthwise_conv_BN_weights,	sizeof(MBConv6_14_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_14_DW_BN_BIAS,				MBConv6_14_depthwise_conv_BN_bias,		sizeof(MBConv6_14_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_14_PRJ_BN_MEAN,      MBConv6_14_project_conv_BN_mean,		  sizeof(MBConv6_14_project_conv_BN_mean) / sizeof(float),
								&MBConv6_14_PRJ_BN_VARIANCE,	MBConv6_14_project_conv_BN_variance,	sizeof(MBConv6_14_project_conv_BN_variance) / sizeof(float),
								&MBConv6_14_PRJ_BN_WEIGHTS,   MBConv6_14_project_conv_BN_weights,		sizeof(MBConv6_14_project_conv_BN_weights) / sizeof(float),
								&MBConv6_14_PRJ_BN_BIAS,			MBConv6_14_project_conv_BN_bias, 		  sizeof(MBConv6_14_project_conv_BN_bias) / sizeof(float));
								
DEFINE_FILTERS_FOR_MBCONV_BN(	
								&MBConv6_15_EXPD_BN_MEAN,     MBConv6_15_expansion_conv_BN_mean,		sizeof(MBConv6_15_expansion_conv_BN_mean) / sizeof(float),
								&MBConv6_15_EXPD_BN_VARIANCE,	MBConv6_15_expansion_conv_BN_variance,sizeof(MBConv6_15_expansion_conv_BN_variance) / sizeof(float),
								&MBConv6_15_EXPD_BN_WEIGHTS,  MBConv6_15_expansion_conv_BN_weights,	sizeof(MBConv6_15_expansion_conv_BN_weights) / sizeof(float),
								&MBConv6_15_EXPD_BN_BIAS,			MBConv6_15_expansion_conv_BN_bias,		sizeof(MBConv6_15_expansion_conv_BN_bias) / sizeof(float),
								&MBConv6_15_DW_BN_MEAN,       MBConv6_15_depthwise_conv_BN_mean,		sizeof(MBConv6_15_depthwise_conv_BN_mean) / sizeof(float),
								&MBConv6_15_DW_BN_VARIANCE,		MBConv6_15_depthwise_conv_BN_variance,sizeof(MBConv6_15_depthwise_conv_BN_variance) / sizeof(float),	
								&MBConv6_15_DW_BN_WEIGHTS,    MBConv6_15_depthwise_conv_BN_weights,	sizeof(MBConv6_15_depthwise_conv_BN_weights) / sizeof(float),
								&MBConv6_15_DW_BN_BIAS,				MBConv6_15_depthwise_conv_BN_bias,		sizeof(MBConv6_15_depthwise_conv_BN_bias) / sizeof(float),
								&MBConv6_15_PRJ_BN_MEAN,      MBConv6_15_project_conv_BN_mean,		  sizeof(MBConv6_15_project_conv_BN_mean) / sizeof(float),
								&MBConv6_15_PRJ_BN_VARIANCE,	MBConv6_15_project_conv_BN_variance,	sizeof(MBConv6_15_project_conv_BN_variance) / sizeof(float),
								&MBConv6_15_PRJ_BN_WEIGHTS,   MBConv6_15_project_conv_BN_weights,		sizeof(MBConv6_15_project_conv_BN_weights) / sizeof(float),
								&MBConv6_15_PRJ_BN_BIAS,			MBConv6_15_project_conv_BN_bias, 		  sizeof(MBConv6_15_project_conv_BN_bias) / sizeof(float));
								

set_allocate_copy_array_Device(&D_STEM_BN_MEAN, Stem_BN_mean,
                sizeof(Stem_BN_mean)/sizeof(float), 1, 1,
                "STEM MEAN"); 
set_allocate_copy_array_Device(&D_STEM_BN_VARIANCE, Stem_BN_variance,
                sizeof(Stem_BN_variance)/sizeof(float), 1, 1,
                "STEAM VARIANCE"); 
set_allocate_copy_array_Device(&D_STEM_BN_WEIGHTS, Stem_BN_weights,
                sizeof(Stem_BN_weights)/sizeof(float), 1, 1,
                "STEM WEIGHTS"); 
set_allocate_copy_array_Device(&D_STEM_BN_BIAS, Stem_BN_bias,
                sizeof(Stem_BN_bias)/sizeof(float), 1, 1,
                "STEM BIAS"); 
                

set_allocate_copy_array_Device(&D_HEAD_BN_MEAN, Head_BN_mean,
                sizeof(Head_BN_mean)/sizeof(float), 1, 1,
                "HEAD MEAN"); 
set_allocate_copy_array_Device(&D_HEAD_BN_VARIANCE, Head_BN_variance,
                sizeof(Head_BN_variance)/sizeof(float), 1, 1,
                "HEAD VARIANCE"); 
set_allocate_copy_array_Device(&D_HEAD_BN_WEIGHTS, Head_BN_weights,
                sizeof(Head_BN_weights)/sizeof(float), 1, 1,
                "HEAD WEIGHTS"); 
set_allocate_copy_array_Device(&D_HEAD_BN_BIAS, Head_BN_bias,
                sizeof(Head_BN_bias)/sizeof(float), 1, 1,
                "HEAD BIAS"); 


start();
  // 3. Move through all layers starting from stem layer till head layer
  Matrix ConvOutStem;
  STEM_LAYER(&DInput_Mat, &F_STEM,
              INPUT_IMAGE_HEIGHT, INPUT_IMAGE_WIDTH, INPUT_IMAGE_DEPTH,
              STEM_FILTER_HEIGHT, STEM_FILTER_WIDTH, STEM_FILTER_DEPTH, STEM_FILTER_DENSITY,
              STEM_PADDING, STEM_STRIDE,
              &ConvOutStem);


  Matrix ConvOut1_0;
  MBCONV1_0_flag = 1;  

  MBConv_Layer(&ConvOutStem, &ConvOut1_0,
                &D_MBConv_1_0_EXPD_WEIGHTS,       &D_MBConv_1_0_DW_WEIGHTS,
                &D_MBConv_1_0_SQZ_1_WEIGHTS,      &D_MBConv_1_0_SQZ_2_WEIGHTS,
                &D_MBConv_1_0_PRJ_WEIGHTS,
                MBCONV_1_0_EXPD_F_DENSITY,        MBCONV_1_0_DW_F_DENSITY, 
                MBCONV_1_0_SQZ_1_F_DENSITY,       MBCONV_1_0_SQZ_2_F_DENSITY, 
                MBCONV_1_0_PRJ_F_DENSITY,
                ConvOutStem.depth,                MBCONV_1_0_PRJ_F_DENSITY, MBCONV_1_0_DW_F_HEIGHT,
                MBCONV_1_0_STRIDE,                MBCONV_1_0_PADDING, MBCONV_1_0_SKIP,
                &MBConv1_0_SQZ_1_bias, 	          &MBConv1_0_SQZ_2_bias,
                NULL, NULL,
                NULL, NULL,
                &MBConv1_0_DW_BN_MEAN,           	&MBConv1_0_DW_BN_VARIANCE,
                &MBConv1_0_DW_BN_WEIGHTS,        	&MBConv1_0_DW_BN_BIAS,
                &MBConv1_0_PRJ_BN_MEAN,           &MBConv1_0_PRJ_BN_VARIANCE,
                &MBConv1_0_PRJ_BN_WEIGHTS,        &MBConv1_0_PRJ_BN_BIAS);
  MBCONV1_0_flag = 0;
  

  Matrix ConvOut;
  MBConv_Layer(&ConvOut1_0, &ConvOut,
                &D_MBConv_6_1_EXPD_WEIGHTS,       &D_MBConv_6_1_DW_WEIGHTS,
                &D_MBConv_6_1_SQZ_1_WEIGHTS,      &D_MBConv_6_1_SQZ_2_WEIGHTS,
                &D_MBConv_6_1_PRJ_WEIGHTS,
                MBCONV_6_1_EXPD_F_DENSITY,        MBCONV_6_1_DW_F_DENSITY, 
                MBCONV_6_1_SQZ_1_F_DENSITY,       MBCONV_6_1_SQZ_2_F_DENSITY, 
                MBCONV_6_1_PRJ_F_DENSITY,
                ConvOut1_0.depth,                 MBCONV_6_1_PRJ_F_DENSITY, MBCONV_6_1_DW_F_HEIGHT,
                MBCONV_6_1_STRIDE,                MBCONV_6_1_PADDING, MBCONV_6_1_SKIP,
                &MBConv6_1_SQZ_1_bias, 	          &MBConv6_1_SQZ_2_bias,
                &MBConv6_1_EXPD_BN_MEAN,          &MBConv6_1_EXPD_BN_VARIANCE,
                &MBConv6_1_EXPD_BN_WEIGHTS,       &MBConv6_1_EXPD_BN_BIAS,
                &MBConv6_1_DW_BN_MEAN,           	&MBConv6_1_DW_BN_VARIANCE,
                &MBConv6_1_DW_BN_WEIGHTS,        	&MBConv6_1_DW_BN_BIAS,
                &MBConv6_1_PRJ_BN_MEAN,           &MBConv6_1_PRJ_BN_VARIANCE,
                &MBConv6_1_PRJ_BN_WEIGHTS,        &MBConv6_1_PRJ_BN_BIAS);


  Matrix ConvOut2;
  MBConv_Layer(&ConvOut, &ConvOut2,
                &D_MBConv_6_2_EXPD_WEIGHTS,       &D_MBConv_6_2_DW_WEIGHTS,
                &D_MBConv_6_2_SQZ_1_WEIGHTS,      &D_MBConv_6_2_SQZ_2_WEIGHTS,
                &D_MBConv_6_2_PRJ_WEIGHTS,
                MBCONV_6_2_EXPD_F_DENSITY,        MBCONV_6_2_DW_F_DENSITY, 
                MBCONV_6_2_SQZ_1_F_DENSITY,       MBCONV_6_2_SQZ_2_F_DENSITY, 
                MBCONV_6_2_PRJ_F_DENSITY,
                ConvOut.depth,                    MBCONV_6_2_PRJ_F_DENSITY, MBCONV_6_2_DW_F_HEIGHT,
                MBCONV_6_2_STRIDE,                MBCONV_6_2_PADDING, MBCONV_6_2_SKIP,
                &MBConv6_2_SQZ_1_bias, 	          &MBConv6_2_SQZ_2_bias,
                &MBConv6_2_EXPD_BN_MEAN,          &MBConv6_2_EXPD_BN_VARIANCE,
                &MBConv6_2_EXPD_BN_WEIGHTS,       &MBConv6_2_EXPD_BN_BIAS,
                &MBConv6_2_DW_BN_MEAN,           	&MBConv6_2_DW_BN_VARIANCE,
                &MBConv6_2_DW_BN_WEIGHTS,        	&MBConv6_2_DW_BN_BIAS,
                &MBConv6_2_PRJ_BN_MEAN,           &MBConv6_2_PRJ_BN_VARIANCE,
                &MBConv6_2_PRJ_BN_WEIGHTS,        &MBConv6_2_PRJ_BN_BIAS); 


  Matrix ConvOut3;
	MBConv_Layer(&ConvOut2, &ConvOut3,
                  &D_MBConv_6_3_EXPD_WEIGHTS,       &D_MBConv_6_3_DW_WEIGHTS,
                  &D_MBConv_6_3_SQZ_1_WEIGHTS,      &D_MBConv_6_3_SQZ_2_WEIGHTS,
                  &D_MBConv_6_3_PRJ_WEIGHTS,
                  MBCONV_6_3_EXPD_F_DENSITY,        MBCONV_6_3_DW_F_DENSITY, 
                  MBCONV_6_3_SQZ_1_F_DENSITY,       MBCONV_6_3_SQZ_2_F_DENSITY, 
                  MBCONV_6_3_PRJ_F_DENSITY,
                  ConvOut2.depth,                   MBCONV_6_3_PRJ_F_DENSITY, MBCONV_6_3_DW_F_HEIGHT,
                  MBCONV_6_3_STRIDE,                MBCONV_6_3_PADDING, MBCONV_6_3_SKIP,
                  &MBConv6_3_SQZ_1_bias,  	        &MBConv6_3_SQZ_2_bias,
									&MBConv6_3_EXPD_BN_MEAN,          &MBConv6_3_EXPD_BN_VARIANCE,
									&MBConv6_3_EXPD_BN_WEIGHTS,       &MBConv6_3_EXPD_BN_BIAS,
									&MBConv6_3_DW_BN_MEAN,           	&MBConv6_3_DW_BN_VARIANCE,
									&MBConv6_3_DW_BN_WEIGHTS,        	&MBConv6_3_DW_BN_BIAS,
									&MBConv6_3_PRJ_BN_MEAN,           &MBConv6_3_PRJ_BN_VARIANCE,
									&MBConv6_3_PRJ_BN_WEIGHTS,        &MBConv6_3_PRJ_BN_BIAS);  
 

  // MBConv6_4 layer implementation

  Matrix ConvOut4;
  MBConv_Layer(&ConvOut3, &ConvOut4,
                &D_MBConv_6_4_EXPD_WEIGHTS,       &D_MBConv_6_4_DW_WEIGHTS,
                &D_MBConv_6_4_SQZ_1_WEIGHTS,      &D_MBConv_6_4_SQZ_2_WEIGHTS,
                &D_MBConv_6_4_PRJ_WEIGHTS,
                MBCONV_6_4_EXPD_F_DENSITY,        MBCONV_6_4_DW_F_DENSITY, 
                MBCONV_6_4_SQZ_1_F_DENSITY,       MBCONV_6_4_SQZ_2_F_DENSITY, 
                MBCONV_6_4_PRJ_F_DENSITY,
                ConvOut3.depth,                   MBCONV_6_4_PRJ_F_DENSITY, MBCONV_6_4_DW_F_HEIGHT,
                MBCONV_6_4_STRIDE,                MBCONV_6_4_PADDING, MBCONV_6_4_SKIP,
                &MBConv6_4_SQZ_1_bias,  	        &MBConv6_4_SQZ_2_bias,
                &MBConv6_4_EXPD_BN_MEAN,          &MBConv6_4_EXPD_BN_VARIANCE,
                &MBConv6_4_EXPD_BN_WEIGHTS,       &MBConv6_4_EXPD_BN_BIAS,
                &MBConv6_4_DW_BN_MEAN,           	&MBConv6_4_DW_BN_VARIANCE,
                &MBConv6_4_DW_BN_WEIGHTS,        	&MBConv6_4_DW_BN_BIAS,
                &MBConv6_4_PRJ_BN_MEAN,           &MBConv6_4_PRJ_BN_VARIANCE,
                &MBConv6_4_PRJ_BN_WEIGHTS,        &MBConv6_4_PRJ_BN_BIAS);   
  

  Matrix ConvOut5;
  MBConv_Layer(&ConvOut4, &ConvOut5,
                &D_MBConv_6_5_EXPD_WEIGHTS,       &D_MBConv_6_5_DW_WEIGHTS,
                &D_MBConv_6_5_SQZ_1_WEIGHTS,      &D_MBConv_6_5_SQZ_2_WEIGHTS,
                &D_MBConv_6_5_PRJ_WEIGHTS,
                MBCONV_6_5_EXPD_F_DENSITY,        MBCONV_6_5_DW_F_DENSITY, 
                MBCONV_6_5_SQZ_1_F_DENSITY,       MBCONV_6_5_SQZ_2_F_DENSITY, 
                MBCONV_6_5_PRJ_F_DENSITY,
                ConvOut4.depth,                   MBCONV_6_5_PRJ_F_DENSITY, MBCONV_6_5_DW_F_HEIGHT,
                MBCONV_6_5_STRIDE,                MBCONV_6_5_PADDING, MBCONV_6_5_SKIP,
                &MBConv6_5_SQZ_1_bias,  	        &MBConv6_5_SQZ_2_bias,
                &MBConv6_5_EXPD_BN_MEAN,          &MBConv6_5_EXPD_BN_VARIANCE,
                &MBConv6_5_EXPD_BN_WEIGHTS,       &MBConv6_5_EXPD_BN_BIAS,
                &MBConv6_5_DW_BN_MEAN,           	&MBConv6_5_DW_BN_VARIANCE,
                &MBConv6_5_DW_BN_WEIGHTS,        	&MBConv6_5_DW_BN_BIAS,
                &MBConv6_5_PRJ_BN_MEAN,           &MBConv6_5_PRJ_BN_VARIANCE,
                &MBConv6_5_PRJ_BN_WEIGHTS,        &MBConv6_5_PRJ_BN_BIAS); 
            


  // MBConv6_6 layer implementation


  Matrix ConvOut6;
  MBConv_Layer(&ConvOut5, &ConvOut6,
                &D_MBConv_6_6_EXPD_WEIGHTS,       &D_MBConv_6_6_DW_WEIGHTS,
                &D_MBConv_6_6_SQZ_1_WEIGHTS,      &D_MBConv_6_6_SQZ_2_WEIGHTS,
                &D_MBConv_6_6_PRJ_WEIGHTS,
                MBCONV_6_6_EXPD_F_DENSITY,        MBCONV_6_6_DW_F_DENSITY, 
                MBCONV_6_6_SQZ_1_F_DENSITY,       MBCONV_6_6_SQZ_2_F_DENSITY, 
                MBCONV_6_6_PRJ_F_DENSITY,
                ConvOut5.depth,                   MBCONV_6_6_PRJ_F_DENSITY, MBCONV_6_6_DW_F_HEIGHT,
                MBCONV_6_6_STRIDE,                MBCONV_6_6_PADDING, MBCONV_6_6_SKIP,
                &MBConv6_6_SQZ_1_bias, 	          &MBConv6_6_SQZ_2_bias,
                &MBConv6_6_EXPD_BN_MEAN,          &MBConv6_6_EXPD_BN_VARIANCE,
                &MBConv6_6_EXPD_BN_WEIGHTS,       &MBConv6_6_EXPD_BN_BIAS,
                &MBConv6_6_DW_BN_MEAN,           	&MBConv6_6_DW_BN_VARIANCE,
                &MBConv6_6_DW_BN_WEIGHTS,        	&MBConv6_6_DW_BN_BIAS,
                &MBConv6_6_PRJ_BN_MEAN,           &MBConv6_6_PRJ_BN_VARIANCE,
                &MBConv6_6_PRJ_BN_WEIGHTS,        &MBConv6_6_PRJ_BN_BIAS);  
            


  // MBConv6_7 layer implementation


  Matrix ConvOut7;
  MBConv_Layer(&ConvOut6, &ConvOut7,
                &D_MBConv_6_7_EXPD_WEIGHTS,       &D_MBConv_6_7_DW_WEIGHTS,
                &D_MBConv_6_7_SQZ_1_WEIGHTS,      &D_MBConv_6_7_SQZ_2_WEIGHTS,
                &D_MBConv_6_7_PRJ_WEIGHTS,
                MBCONV_6_7_EXPD_F_DENSITY,        MBCONV_6_7_DW_F_DENSITY, 
                MBCONV_6_7_SQZ_1_F_DENSITY,       MBCONV_6_7_SQZ_2_F_DENSITY, 
                MBCONV_6_7_PRJ_F_DENSITY,
                ConvOut6.depth,                   MBCONV_6_7_PRJ_F_DENSITY, MBCONV_6_7_DW_F_HEIGHT,                   
                MBCONV_6_7_STRIDE,                MBCONV_6_7_PADDING, MBCONV_6_7_SKIP,
                &MBConv6_7_SQZ_1_bias,  	        &MBConv6_7_SQZ_2_bias,
                &MBConv6_7_EXPD_BN_MEAN,          &MBConv6_7_EXPD_BN_VARIANCE,
                &MBConv6_7_EXPD_BN_WEIGHTS,       &MBConv6_7_EXPD_BN_BIAS,
                &MBConv6_7_DW_BN_MEAN,           	&MBConv6_7_DW_BN_VARIANCE,
                &MBConv6_7_DW_BN_WEIGHTS,        	&MBConv6_7_DW_BN_BIAS,
                &MBConv6_7_PRJ_BN_MEAN,           &MBConv6_7_PRJ_BN_VARIANCE,
                &MBConv6_7_PRJ_BN_WEIGHTS,        &MBConv6_7_PRJ_BN_BIAS);  
          


  // MBConv6_8 layer implementation


  Matrix ConvOut8;
  MBConv_Layer(&ConvOut7, &ConvOut8,
                &D_MBConv_6_8_EXPD_WEIGHTS,       &D_MBConv_6_8_DW_WEIGHTS,
                &D_MBConv_6_8_SQZ_1_WEIGHTS,      &D_MBConv_6_8_SQZ_2_WEIGHTS,
                &D_MBConv_6_8_PRJ_WEIGHTS,
                MBCONV_6_8_EXPD_F_DENSITY,        MBCONV_6_8_DW_F_DENSITY, 
                MBCONV_6_8_SQZ_1_F_DENSITY,       MBCONV_6_8_SQZ_2_F_DENSITY, 
                MBCONV_6_8_PRJ_F_DENSITY,
                ConvOut7.depth,                   MBCONV_6_8_PRJ_F_DENSITY, MBCONV_6_8_DW_F_HEIGHT,    
                MBCONV_6_8_STRIDE,                MBCONV_6_8_PADDING, MBCONV_6_8_SKIP,
                &MBConv6_8_SQZ_1_bias,  	        &MBConv6_8_SQZ_2_bias,
                &MBConv6_8_EXPD_BN_MEAN,          &MBConv6_8_EXPD_BN_VARIANCE,
                &MBConv6_8_EXPD_BN_WEIGHTS,       &MBConv6_8_EXPD_BN_BIAS,
                &MBConv6_8_DW_BN_MEAN,           	&MBConv6_8_DW_BN_VARIANCE,
                &MBConv6_8_DW_BN_WEIGHTS,        	&MBConv6_8_DW_BN_BIAS,
                &MBConv6_8_PRJ_BN_MEAN,           &MBConv6_8_PRJ_BN_VARIANCE,
                &MBConv6_8_PRJ_BN_WEIGHTS,        &MBConv6_8_PRJ_BN_BIAS); 
        


  // MBConv6_9 layer implementation


  Matrix ConvOut9;
  MBConv_Layer(&ConvOut8, &ConvOut9,
                &D_MBConv_6_9_EXPD_WEIGHTS,       &D_MBConv_6_9_DW_WEIGHTS,
                &D_MBConv_6_9_SQZ_1_WEIGHTS,      &D_MBConv_6_9_SQZ_2_WEIGHTS,
                &D_MBConv_6_9_PRJ_WEIGHTS,
                MBCONV_6_9_EXPD_F_DENSITY,        MBCONV_6_9_DW_F_DENSITY, 
                MBCONV_6_9_SQZ_1_F_DENSITY,       MBCONV_6_9_SQZ_2_F_DENSITY, 
                MBCONV_6_9_PRJ_F_DENSITY,
                ConvOut8.depth,                   MBCONV_6_9_PRJ_F_DENSITY, MBCONV_6_9_DW_F_HEIGHT,
                MBCONV_6_9_STRIDE,                MBCONV_6_9_PADDING, MBCONV_6_9_SKIP,
                &MBConv6_9_SQZ_1_bias,  	        &MBConv6_9_SQZ_2_bias,
                &MBConv6_9_EXPD_BN_MEAN,          &MBConv6_9_EXPD_BN_VARIANCE,
                &MBConv6_9_EXPD_BN_WEIGHTS,       &MBConv6_9_EXPD_BN_BIAS,
                &MBConv6_9_DW_BN_MEAN,           	&MBConv6_9_DW_BN_VARIANCE,
                &MBConv6_9_DW_BN_WEIGHTS,        	&MBConv6_9_DW_BN_BIAS,
                &MBConv6_9_PRJ_BN_MEAN,           &MBConv6_9_PRJ_BN_VARIANCE,
                &MBConv6_9_PRJ_BN_WEIGHTS,        &MBConv6_9_PRJ_BN_BIAS);  				  



  // MBConv6_10 layer implementation


  Matrix ConvOut10;
	MBConv_Layer(&ConvOut9, &ConvOut10,
                &D_MBConv_6_10_EXPD_WEIGHTS,      &D_MBConv_6_10_DW_WEIGHTS,
                &D_MBConv_6_10_SQZ_1_WEIGHTS,     &D_MBConv_6_10_SQZ_2_WEIGHTS,
                &D_MBConv_6_10_PRJ_WEIGHTS,
                MBCONV_6_10_EXPD_F_DENSITY,       MBCONV_6_10_DW_F_DENSITY, 
                MBCONV_6_10_SQZ_1_F_DENSITY,      MBCONV_6_10_SQZ_2_F_DENSITY, 
                MBCONV_6_10_PRJ_F_DENSITY,
                ConvOut9.depth,                   MBCONV_6_10_PRJ_F_DENSITY, MBCONV_6_10_DW_F_HEIGHT,
                MBCONV_6_10_STRIDE,               MBCONV_6_10_PADDING, MBCONV_6_10_SKIP,
                &MBConv6_10_SQZ_1_bias, 	        &MBConv6_10_SQZ_2_bias,
                &MBConv6_10_EXPD_BN_MEAN,         &MBConv6_10_EXPD_BN_VARIANCE,
                &MBConv6_10_EXPD_BN_WEIGHTS,      &MBConv6_10_EXPD_BN_BIAS,
                &MBConv6_10_DW_BN_MEAN,           &MBConv6_10_DW_BN_VARIANCE,
                &MBConv6_10_DW_BN_WEIGHTS,        &MBConv6_10_DW_BN_BIAS,
                &MBConv6_10_PRJ_BN_MEAN,          &MBConv6_10_PRJ_BN_VARIANCE,
                &MBConv6_10_PRJ_BN_WEIGHTS,       &MBConv6_10_PRJ_BN_BIAS);   
  


  // MBConv6_11 layer implementation


  Matrix ConvOut11;
  MBConv_Layer(&ConvOut10, &ConvOut11,
                &D_MBConv_6_11_EXPD_WEIGHTS,    &D_MBConv_6_11_DW_WEIGHTS,
                &D_MBConv_6_11_SQZ_1_WEIGHTS,   &D_MBConv_6_11_SQZ_2_WEIGHTS,
                &D_MBConv_6_11_PRJ_WEIGHTS,
                MBCONV_6_11_EXPD_F_DENSITY,     MBCONV_6_11_DW_F_DENSITY, 
                MBCONV_6_11_SQZ_1_F_DENSITY,    MBCONV_6_11_SQZ_2_F_DENSITY, 
                MBCONV_6_11_PRJ_F_DENSITY,  
                ConvOut10.depth,                MBCONV_6_11_PRJ_F_DENSITY, MBCONV_6_11_DW_F_HEIGHT,
                MBCONV_6_11_STRIDE,             MBCONV_6_11_PADDING, MBCONV_6_11_SKIP,
                &MBConv6_11_SQZ_1_bias,         &MBConv6_11_SQZ_2_bias,
                &MBConv6_11_EXPD_BN_MEAN,       &MBConv6_11_EXPD_BN_VARIANCE,
                &MBConv6_11_EXPD_BN_WEIGHTS,    &MBConv6_11_EXPD_BN_BIAS,
                &MBConv6_11_DW_BN_MEAN,         &MBConv6_11_DW_BN_VARIANCE,
                &MBConv6_11_DW_BN_WEIGHTS,      &MBConv6_11_DW_BN_BIAS,
                &MBConv6_11_PRJ_BN_MEAN,        &MBConv6_11_PRJ_BN_VARIANCE,
                &MBConv6_11_PRJ_BN_WEIGHTS,     &MBConv6_11_PRJ_BN_BIAS);  
  


  // MBConv6_12 layer implementation


  Matrix ConvOut12;
  MBConv_Layer(&ConvOut11, &ConvOut12,
                &D_MBConv_6_12_EXPD_WEIGHTS,  &D_MBConv_6_12_DW_WEIGHTS,
                &D_MBConv_6_12_SQZ_1_WEIGHTS, &D_MBConv_6_12_SQZ_2_WEIGHTS,
                &D_MBConv_6_12_PRJ_WEIGHTS,
                MBCONV_6_12_EXPD_F_DENSITY,   MBCONV_6_12_DW_F_DENSITY, 
                MBCONV_6_12_SQZ_1_F_DENSITY,  MBCONV_6_12_SQZ_2_F_DENSITY, 
                MBCONV_6_12_PRJ_F_DENSITY,
                ConvOut11.depth,              MBCONV_6_12_PRJ_F_DENSITY, MBCONV_6_12_DW_F_HEIGHT,
                MBCONV_6_12_STRIDE,           MBCONV_6_12_PADDING, MBCONV_6_12_SKIP,
                &MBConv6_12_SQZ_1_bias,       &MBConv6_12_SQZ_2_bias,
                &MBConv6_12_EXPD_BN_MEAN,     &MBConv6_12_EXPD_BN_VARIANCE,
                &MBConv6_12_EXPD_BN_WEIGHTS,  &MBConv6_12_EXPD_BN_BIAS,
                &MBConv6_12_DW_BN_MEAN,       &MBConv6_12_DW_BN_VARIANCE,
                &MBConv6_12_DW_BN_WEIGHTS,    &MBConv6_12_DW_BN_BIAS,
                &MBConv6_12_PRJ_BN_MEAN,      &MBConv6_12_PRJ_BN_VARIANCE,
                &MBConv6_12_PRJ_BN_WEIGHTS,   &MBConv6_12_PRJ_BN_BIAS);   
  


  // MBConv6_13 layer implementation

  Matrix ConvOut13;
  MBConv_Layer(&ConvOut12, &ConvOut13,
                &D_MBConv_6_13_EXPD_WEIGHTS,    &D_MBConv_6_13_DW_WEIGHTS,
                &D_MBConv_6_13_SQZ_1_WEIGHTS,   &D_MBConv_6_13_SQZ_2_WEIGHTS,
                &D_MBConv_6_13_PRJ_WEIGHTS,
                MBCONV_6_13_EXPD_F_DENSITY,     MBCONV_6_13_DW_F_DENSITY, 
                MBCONV_6_13_SQZ_1_F_DENSITY,    MBCONV_6_13_SQZ_2_F_DENSITY, 
                MBCONV_6_13_PRJ_F_DENSITY,
                ConvOut12.depth,                MBCONV_6_13_PRJ_F_DENSITY, MBCONV_6_13_DW_F_HEIGHT,
                MBCONV_6_13_STRIDE,             MBCONV_6_13_PADDING, MBCONV_6_13_SKIP,
                &MBConv6_13_SQZ_1_bias,         &MBConv6_13_SQZ_2_bias,
                &MBConv6_13_EXPD_BN_MEAN,       &MBConv6_13_EXPD_BN_VARIANCE,
                &MBConv6_13_EXPD_BN_WEIGHTS,    &MBConv6_13_EXPD_BN_BIAS,
                &MBConv6_13_DW_BN_MEAN,         &MBConv6_13_DW_BN_VARIANCE,
                &MBConv6_13_DW_BN_WEIGHTS,      &MBConv6_13_DW_BN_BIAS,
                &MBConv6_13_PRJ_BN_MEAN,        &MBConv6_13_PRJ_BN_VARIANCE,
                &MBConv6_13_PRJ_BN_WEIGHTS,     &MBConv6_13_PRJ_BN_BIAS);


  Matrix ConvOut14;
  MBConv_Layer(&ConvOut13, &ConvOut14,
                &D_MBConv_6_14_EXPD_WEIGHTS,  &D_MBConv_6_14_DW_WEIGHTS,
                &D_MBConv_6_14_SQZ_1_WEIGHTS, &D_MBConv_6_14_SQZ_2_WEIGHTS,
                &D_MBConv_6_14_PRJ_WEIGHTS,
                MBCONV_6_14_EXPD_F_DENSITY,   MBCONV_6_14_DW_F_DENSITY, 
                MBCONV_6_14_SQZ_1_F_DENSITY,  MBCONV_6_14_SQZ_2_F_DENSITY, 
                MBCONV_6_14_PRJ_F_DENSITY,
                ConvOut13.depth,              MBCONV_6_14_PRJ_F_DENSITY, MBCONV_6_14_DW_F_HEIGHT,
                MBCONV_6_14_STRIDE,           MBCONV_6_14_PADDING, MBCONV_6_14_SKIP,
                &MBConv6_14_SQZ_1_bias, 	    &MBConv6_14_SQZ_2_bias,
                &MBConv6_14_EXPD_BN_MEAN,     &MBConv6_14_EXPD_BN_VARIANCE,
                &MBConv6_14_EXPD_BN_WEIGHTS,  &MBConv6_14_EXPD_BN_BIAS,
                &MBConv6_14_DW_BN_MEAN,       &MBConv6_14_DW_BN_VARIANCE,
                &MBConv6_14_DW_BN_WEIGHTS,    &MBConv6_14_DW_BN_BIAS,
                &MBConv6_14_PRJ_BN_MEAN,      &MBConv6_14_PRJ_BN_VARIANCE,
                &MBConv6_14_PRJ_BN_WEIGHTS,   &MBConv6_14_PRJ_BN_BIAS);  


  Matrix ConvOut15;
  MBConv_Layer(&ConvOut14, &ConvOut15,
                &D_MBConv_6_15_EXPD_WEIGHTS,  &D_MBConv_6_15_DW_WEIGHTS,
                &D_MBConv_6_15_SQZ_1_WEIGHTS, &D_MBConv_6_15_SQZ_2_WEIGHTS,
                &D_MBConv_6_15_PRJ_WEIGHTS,
                MBCONV_6_15_EXPD_F_DENSITY,   MBCONV_6_15_DW_F_DENSITY, 
                MBCONV_6_15_SQZ_1_F_DENSITY,  MBCONV_6_15_SQZ_2_F_DENSITY, 
                MBCONV_6_15_PRJ_F_DENSITY,
                ConvOut14.depth,              MBCONV_6_15_PRJ_F_DENSITY, MBCONV_6_15_DW_F_HEIGHT,
                MBCONV_6_15_STRIDE,           MBCONV_6_15_PADDING, MBCONV_6_15_SKIP,
                &MBConv6_15_SQZ_1_bias,       &MBConv6_15_SQZ_2_bias,
                &MBConv6_15_EXPD_BN_MEAN,     &MBConv6_15_EXPD_BN_VARIANCE,
                &MBConv6_15_EXPD_BN_WEIGHTS,  &MBConv6_15_EXPD_BN_BIAS,
                &MBConv6_15_DW_BN_MEAN,       &MBConv6_15_DW_BN_VARIANCE,
                &MBConv6_15_DW_BN_WEIGHTS,    &MBConv6_15_DW_BN_BIAS,
                &MBConv6_15_PRJ_BN_MEAN,      &MBConv6_15_PRJ_BN_VARIANCE,
                &MBConv6_15_PRJ_BN_WEIGHTS,   &MBConv6_15_PRJ_BN_BIAS);   

  // Head layer
  Matrix HEAD_OUT;
  HEAD_LAYER(&ConvOut15, &HEAD_CONV_WEIGHTS, &HEAD_FC_WEIGHTS,
              HEAD_CONV_F_HEIGHT, HEAD_CONV_F_WIDTH, HEAD_CONV_F_DEPTH, HEAD_CONV_F_DENSITY,
              0, 1,
              &HEAD_OUT);

  stop("Model: ", 0);
}



// The last layer in efficient net
void HEAD_LAYER(Matrix *INPUT_MATRIX, Matrix *F_HEAD, Matrix *FC_WEIGHTS,
                int filter_height, int filter_width, int filter_depth, int filter_density,
                int padding, int stride,
                Matrix *HEAD_OUT)
{                
  // Calculate output dimensions       
  int out_height = (INPUT_MATRIX -> height + 2 * padding - filter_height) / stride + 1;
  int out_width = (INPUT_MATRIX -> width + 2 * padding - filter_width) / stride + 1;
  int out_depth = filter_density;

  Set_DeviceMatrix(out_height, out_width, out_depth, HEAD_OUT,
                   "Output is allocated in device memory"); 

  // 1st 3 layers: Conv2d 1x1: BN: Swish()
  Conv2d_Layer(INPUT_MATRIX,  F_HEAD, HEAD_OUT,
              stride, padding,
              INPUT_MATRIX -> depth, out_depth, filter_density,
              Conv2d_1_x_1, NO_ACTIVATION,
              0, NULL);
 
  BN_ALL_PRE_DEFINED(HEAD_OUT, SWISH_ACTIVATION, 
                      &D_HEAD_BN_MEAN,	&D_HEAD_BN_VARIANCE ,
                      &D_HEAD_BN_WEIGHTS, &D_HEAD_BN_BIAS);


  // 4th layer: Average pooling layer which is just a reduction sum layer
  // Get mean values for all channels; Dims(1 x 1 x InputDepth)
  
  Matrix MEAN, Result_Mean;

  Set_DeviceMatrix(HEAD_OUT -> depth,
                    (int)ceil((double)HEAD_OUT -> height * HEAD_OUT -> width / (2 * BLOCK_SIZE)),
                    1, 
                    &Result_Mean, 
                    "Reesult Mean matrix allocated in device memory");

  REDUCTION_SUM(HEAD_OUT, &MEAN, &Result_Mean);


  // 5th layer: Fully connected layer::

  // Set Output matrix details
  Matrix Out1;
  Set_DeviceMatrix(1, 1000, 1, &Out1, "Setting Final Model Output matrix in device memory");
     
  Conv_vidMultiplier(&Out1, FC_WEIGHTS, &Result_Mean,
                      1, 1000, 1,
                      Conv2d_1_x_1, 1,
                      NO_ACTIVATION, 
                      0, NULL);
  
  Matrix tmp_out_host;
  set_allocate_Host(&tmp_out_host, 1, 1000, 1);
  just_copy_DTH(&tmp_out_host, &Out1, "Copying to add bias");
 
  for (int i = 0; i < 1000; i++)
  {
    tmp_out_host.elements[i] += Head_linear_bias[i];
  }
/*
  just_copy_HTD(&Out1, &tmp_out_host, "Copying to add bias");
  show_me_enhanced_from_devince(&Out1, "Model final output::");*/
}

// The first layer in efficient net: 
// It reutnrs a pointer to matrix, its elements are allocated in device memory 
void STEM_LAYER(Matrix *DInput_Mat, Matrix *F_STEM,
                  int image_height, int image_width, int image_depth,
                  int filter_height, int filter_width, int filter_depth, int filter_density,
                  int padding, int stride,
                  Matrix *STEM_OUT)
{

  // Calculate output dimensions       
  int out_height = (image_height + 2 * padding - filter_height) / stride + 1;
  int out_width = (image_width + 2 * padding - filter_width) / stride + 1;
  int out_depth = filter_density;
 

  // Allow the output from this layer to go accross the next layer       
  Set_DeviceMatrix(out_height, out_width, out_depth, STEM_OUT,
                   "Output is allocated in device memory"); 
 
  Conv2d_Layer(DInput_Mat,  F_STEM, STEM_OUT,
              stride, padding,
              image_depth, out_depth, filter_density,
              Regular_Conv, NO_ACTIVATION,
              0, NULL);
 

  BN_ALL_PRE_DEFINED(STEM_OUT, SWISH_ACTIVATION, 
                      &D_STEM_BN_MEAN, &D_STEM_BN_VARIANCE ,
                      &D_STEM_BN_WEIGHTS, &D_STEM_BN_BIAS);  
}

'File written in /content/src/APP.cu'

In [81]:
%%cuda --name FUNCTIONS.cu 

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime_api.h>
#include "device_launch_parameters.h"
#include <cusolverDn.h>
#include <cuda_runtime.h>


#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/functionsV2.h"
#include "/content/drive/MyDrive/Colab_Notebooks/MBCONVS_float/KERNELSH.h"

static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))

float time_defined = 0, tmp_time = 0, total_time_for_layer = 0;; 
cudaEvent_t start_timing, stop_timing;


int show_out = 0;

int total_constant_memory = 0;
                
// Device memory for filters
void DEFINE_FILTERS_FOR_MBCONV(Matrix *D_f1, float *filter1, int h1, int w1, int dens1,
                               Matrix *D_f2, float *filter2, int h2, int w2, int dens2,
                               Matrix *D_f3, float *filter3, int h3, int w3, int dens3,
                               Matrix *D_f4, float *filter4, int h4, int w4, int dens4,
                               Matrix *D_f5, float *filter5, int h5, int w5, int dens5)
{
    // Note: No allocations are done, just pointers point to matrices pre-defined

    // This condition is important as the float * is NULL
    if (MBCONV1_0_flag == 1);
    else
      set_allocate_copy_array_Device(D_f1, filter1,
                                    h1, w1, dens1,
                                    "1st filter allocated");
 
    set_allocate_copy_array_Device(D_f2, filter2,
                                    h2, w2, dens2,
                                    "2nd filter allocated");
 
    set_allocate_copy_array_Device(D_f3, filter3,
                                    h3, w3, dens3,
                                    "3rd filter allocated");

    set_allocate_copy_array_Device(D_f4, filter4,
                                    h4, w4, dens4,
                                    "4th filter allocated");

    set_allocate_copy_array_Device(D_f5, filter5,
                                    h5, w5, dens5,
                                    "5th filter allocated");                                                         
}

// Free the device filters
void FREE_FILTERS_FOR_MBCONV(Matrix *D_f1, Matrix *D_f2, 
                             Matrix *D_f3, Matrix *D_f4,
                             Matrix *D_f5)
{
  cudaFree(D_f1 -> elements);
  cudaFree(D_f2 -> elements);
  cudaFree(D_f3 -> elements);
  cudaFree(D_f4 -> elements);
  cudaFree(D_f5 -> elements);
}

void REDUCTION_SUM(Matrix* Output_Modified, Matrix *sum, Matrix *DMean)
{
    /*
      The mean will be a row vector of 1 x C;
      where C is number of original matrix channels
      All input matrices for this function are device matrices,
      except for sum, it's just a transition that later can be removed 
    */

    // Define number of blocks in different directions
    int nbx = 0;
    int nby = 0;
    int nbz = 1;

    size_t size;
    cudaError err;
 
    /*
      Load input Matrix inot device t calculate mean for it
      Unfortionately the code requires to copy the input matrix
    */
 
    Matrix DInputMat;
    // Allocate and set its dimensions as needed from the algorithm
    Set_DeviceMatrix(Output_Modified -> depth, Output_Modified -> height * Output_Modified -> width, 1,
                     &DInputMat, "Copyting Input of reduction mean function into Device memory");

    // Copy input from a device memory to device memory in order to process the elements
    size = DInputMat.height * DInputMat.width * DInputMat.depth * sizeof(float);
    err = cudaMemcpy(DInputMat.elements, Output_Modified -> elements, size, cudaMemcpyDeviceToDevice);
    CheckCudaError("Copying mean matrix elements from ", err);

    /* Starting reduction sum calculations */

    // Note: All blocks are 1D threads. We use Block.x to reduce 1 channel elements'
    // Diffferent block.y to address different number of channels
    nbx = (int)ceil((double)DInputMat.width / (2 * BLOCK_SIZE));
    nby = (int)ceil((double)DInputMat.height);

    if (nbx == 0) nbx = 1;
    if (nby == 0) nby = 1;

    // For loop is held to maintain huge number of summations needed
    for (int i = 0; DInputMat.width != 1; i++)
    {
        dim3 dim_Grid2(nbx, nby, nbz);
        dim3 dim_Block2(BLOCK_SIZE, 1, 1);

        // Make sure to synch between multiple runs
       //cudaDeviceSynchronize();
        
        BN_Kernel_Mean_Reduction <<< dim_Grid2, dim_Block2 >>> (DInputMat.elements,
                                                                DInputMat.height,
                                                                DInputMat.width,
                                                                DInputMat.depth,
                                                                DMean -> elements,
                                                                DMean -> width);

        
        // Save and copy mean values array into the filter array
        size = DMean -> height * DMean -> width * DMean -> depth * sizeof(float);
        err = cudaMemcpy(DInputMat.elements, DMean -> elements, size, cudaMemcpyDeviceToDevice);
        CheckCudaError("Copying mean matrix elements from ", err);

        // Modify filter width to fit into the new elements width
        DInputMat.width = nbx;
        DInputMat.height = nby;

        // Recalculate number of blocks in x direction
        nbx = (int)ceil((double)DInputMat.width / (2 * BLOCK_SIZE));
        nby = (int)ceil((double)DInputMat.height);

        if (nbx == 0) nbx = 1;
        if (nby == 0) nby = 1;

        // Set width of mean matrix to the new number of blocks
        DMean -> width = nbx;
        DMean -> height = nby;
    }

    
    // Set mean matrix to 1 X C X 1 to ease further calculations
    DMean -> height = 1; DMean -> width = Output_Modified -> depth; DMean -> depth = 1;

    // Set the results back into the sum matrix to update the pointer
    set_allocate_Host(sum, DMean -> height, DMean -> width, DMean -> depth);
    
    // Copy elements from device to host
    just_copy_DTH(sum, DMean, "Mean matrix copied back to host");
    
    /* Mean calculation are done for all channels and the result is 1D row vector */

    // Divide by number of pixels in 1 channel
    int Divide_Mean_BY_ME = Output_Modified->height * Output_Modified->width;

    // Divide all elements by Number of pixels
    for (int i = 0; i < DMean -> width; i++)
    {
        sum -> elements[i] = sum -> elements[i] / Divide_Mean_BY_ME;
    }
	
    
    just_copy_HTD(DMean, sum, "Mean matrix copied back to Device");
 
   
}


/*  Squeeze_and_Excite(&tmp2, &SE_OUT, F3, F4,
                      FD4, FD3, FD4, FD3);
                      */
// Note: input and output channels args represents the 2 Conv layers output channels respectively
void Squeeze_and_Excite(Matrix* InputIMG, Matrix* Result,
                        Matrix* Filter1, Matrix* Filter2,
                        int FilterDensity2, int FilterDensity1,
                        int input_channels, int output_channels,
                        Matrix * First_bias, Matrix *Second_bias)
{
    /*
       Steps in squeeze and excite layer:
        1. Get mean value for a tensor
        2. pass the mean to the covolution, swish, convolution, sigmoid
        3. the result will be a 1 x 1 x C, multiply elementwise.
          "each element in a channel is multiplied by the result's corresponding channel element"
        
        Filter Density means #filters used
 
      Note: All input matrices are device allocated matrices
    */

 
    /*
      Get mean values for all channels; Dims(1 x InputDepth x 1) 
      Note: Mean matrix is a host allocated memory in REDUCTION_SUM;
            It's used to get the final summation from device and
            then divide each element sequentially by total number 
            of elements. It's then later copied back to Result_Mean
            Matrix which is a device matrix.
            "This can be later changed"
    */
 
    Matrix MEAN, Result_Mean;

    Set_DeviceMatrix(InputIMG -> depth,
                      (int)ceil((double)InputIMG -> height * InputIMG -> width / (2 * BLOCK_SIZE)),
                      1, 
                      &Result_Mean, 
                      "Reesult Mean matrix allocated in device memory");

    REDUCTION_SUM(InputIMG, &MEAN, &Result_Mean);
 

    // Tmp1 is used as a transition between 2 convolution layers; Dims(1 x 1 x FilterDensity3)
    Matrix tmp1;
    Set_DeviceMatrix(1, 1, FilterDensity1, &tmp1, "Allocating tmp1 in device for transition");
 
    // tmp2 matrix is the result from sigmoid function: Dims(1 x 1 x FilterDensity4)
    Matrix tmp2;
    Set_DeviceMatrix( 1, 1, FilterDensity2, &tmp2, "Allocating tmp2 in device for final output");
 
    // Sequence: Conv1x1, swish, Conv1x1, sigmoid 
    // Warning: Remember to pre-process Result_Mean matrix to match 1 x 1 x C as it's the input in this case to Conv2d
    Set_HostMatrix(1, 1, InputIMG -> depth, &Result_Mean);
 
    Conv2d_Layer(&Result_Mean, Filter1, &tmp1, 1, 0, input_channels, output_channels, FilterDensity1,
                 Conv2d_1_x_1, SWISH_ACTIVATION,
                 BIASED, First_bias);
    
    Conv2d_Layer(&tmp1, Filter2, &tmp2, 1, 0, output_channels, input_channels, FilterDensity2,
                 Conv2d_1_x_1, SIGMOID_ACTIVATION,
                 BIASED, Second_bias);
 

    int nbx = (int)ceil((double)InputIMG -> width / Tile_GEMM);
    int nby = (int)ceil((double)InputIMG -> height / Tile_GEMM);
    int nbz = InputIMG -> depth;

    if (nbx == 0) nbx = 1;

    if (nby == 0) nby = 1;

    // This is the only kernel that runs 3d Grid; 
    // Each block in z dimension controls 1 channel  
    dim3 dim_Grid2(nbx, nby, nbz);
    dim3 dim_Block2(Tile_GEMM, Tile_GEMM, 1);

    // C then D, the final multiplication is in C matrix
    //cudaDeviceSynchronize();
 
    ConvChannelElementWiseMultiplication <<< dim_Grid2, dim_Block2 >>> (InputIMG -> elements,
                                                                        InputIMG -> height,
                                                                        InputIMG -> width,
                                                                        InputIMG -> depth,
                                                                        tmp2.elements);
 
 
    //cudaDeviceSynchronize();  

   
    cudaFree(tmp1.elements);
    cudaFree(tmp2.elements);
}

// Warning: Fuction Input matrices are allocated in device memory directly
// InputIMG, FilterK and ConvOut are device memory allocations
void Conv2d_Layer(Matrix* InputIMG, Matrix* FilterK, Matrix* ConvOut,
                  int stride, int padding,
                  int InputChannels, int OutputChannels, int FilterDensity,
                  int Conv_Type, int activation_type,
                  int BIASED_CHOISE, Matrix *biasMat)
{
    //printf("The start of Conv2d layer\n\n");
    
    int OutputHeight = 0, OutputWidth = 0, OutputDepth = 0;

    // 1x1 Conv2d is a special case of Convolution
    if (Conv_Type == Conv2d_1_x_1)
    {
        // Conv2d 1x1 has stride = 1, no padding and K = 1

        /*
          Input Dimensions is the same as Output dimensions
          Only Depth of the output channels differ from input
        */
        OutputHeight = InputIMG -> height; OutputWidth = InputIMG -> width; OutputDepth = FilterDensity;

        /*
          Note: Set_HostMatrix function just changes the dimensions
                so it's okey to use on a device memory
        */
     
        // Modify Filter Matrix to have dimensions ((K^2 * M) x C x 1); K = 1
        Set_HostMatrix(1 * 1 * FilterDensity, InputIMG -> depth, 1, FilterK);

        // Modify Input matrix to have dimensions (C x (H * W) x 1)
        Set_HostMatrix(InputIMG -> depth, InputIMG -> height * InputIMG -> width, 1, InputIMG);

        // Modify Output Matrix preprocessing to have dimesions ((K^2 * M) x (H * W) x 1); K = 1
        Set_HostMatrix(1 * 1 * FilterDensity, OutputWidth * OutputHeight, 1, ConvOut);

        Conv_vidMultiplier(ConvOut, InputIMG, FilterK,
                            OutputHeight, OutputWidth, OutputDepth,
                            Conv2d_1_x_1, 1,
                            activation_type, 
                            BIASED_CHOISE, biasMat);
    }
    else if (Conv_Type == DWConv_k_x_k)
    {
        // Ptr is used to alternate between input image and padding if needed
        Matrix* ptr = InputIMG;

        // DWConv2d has stride = s, padding = p and kernel = k
        OutputHeight = ConvOut -> height; OutputWidth = ConvOut -> width; OutputDepth = ConvOut -> depth;

        Matrix padded_matr;
        if (padding != 0)
        {
            Padding_Zeros_Function(InputIMG, padding, &padded_matr);
            ptr = &padded_matr;
        }
       
        Conv_vidMultiplier(ConvOut, ptr, FilterK,
                            OutputHeight, OutputWidth, OutputDepth,
                            DWConv_k_x_k, stride,
                            activation_type, 
                            BIASED_CHOISE, biasMat);

        // Padded matrix is no longer needed as Convout has the final result
    }
    // Any other kernel size goes here
    else
    {        
        // Regular convolution: Filter and input unrolling
        Matrix* ptr = InputIMG;
        OutputHeight = (ptr -> height + 2 * padding - FilterK -> height) / stride + 1;
        OutputWidth = (ptr -> width + 2 * padding - FilterK -> width) / stride + 1;
        OutputDepth = FilterDensity;

        Matrix padded_matr;
        if (padding != 0)
        {
            Padding_Zeros_Function(InputIMG, padding, &padded_matr);
            ptr = &padded_matr;          
        }

        // 1st phase: Filter unrolling

        // Unrolled filter has dimesnios (M x (C * k * k) x 1)
        Set_HostMatrix(FilterDensity, FilterK -> depth / FilterDensity * FilterK -> height * FilterK -> width,
                      1, FilterK);

        // 2nd phase: Input unrolling

        // The unrolled Input matrix has dimensions((C * k * k) x (H_out * W_out) x 1)
        Matrix INPUT_MODIFIED;
        
        Set_DeviceMatrix(ptr -> depth * 3 * 3,
                        OutputHeight * OutputWidth, 1,
                        &INPUT_MODIFIED, 
                        "Input unrolled Matrix allocated in device memory");

        Input_Unroll_gpu(stride, ptr, &INPUT_MODIFIED, OutputHeight, OutputWidth, 3);

        // Convolution output has dimensions of (M x (H_out * W_out) x 1)
        Set_HostMatrix(FilterDensity, OutputWidth * OutputHeight, 1, ConvOut);

          
        // Perform Multiplication and re-edit the dimensions of output
        Conv_vidMultiplier(ConvOut, &INPUT_MODIFIED, FilterK,
                            OutputHeight, OutputWidth, OutputDepth,
                            Regular_Conv, stride,
                            activation_type,
                            BIASED_CHOISE, biasMat);

    }
 }

// 5 Filters needed to run the 4 layers sequentially
void MBConv_Layer(Matrix* Input, Matrix* MBConvOut,
    Matrix* F1, Matrix* F2, Matrix* F3, Matrix* F4, Matrix* F5,
    int FD1, int FD2, int FD3, int FD4, int FD5,
    int input_channels, int output_channels, int FilterSizeDW,
    int Stride, int padding, int skip,
    Matrix *bias1, Matrix *bias2,
    Matrix *MBConv_expansion_conv_BN_mean,     Matrix *MBConv_expansion_conv_BN_variance,
    Matrix *MBConv_expansion_conv_BN_weights,  Matrix *MBConv_expansion_conv_BN_bias,
    Matrix *MBConv_depthwise_conv_BN_mean,     Matrix *MBConv_depthwise_conv_BN_variance,
    Matrix *MBConv_depthwise_conv_BN_weights,  Matrix *MBConv_depthwise_conv_BN_bias,
    Matrix *MBConv_project_conv_BN_mean,       Matrix *MBConv_project_conv_BN_variance,
    Matrix *MBConv_project_conv_BN_weights,    Matrix *MBConv_project_conv_BN_bias)
{
    /*
      Note: MBConv1_0 doesn't have the expansion conv function;
            The input matrices to this function are device matrices;
            including all the filters, you don't need to allocate or
            copy any thing; just pass to the functions
    */

    /*
      ptr_mat is the pointer that gets past expansion conv;
      Meaning: in case of MBconv1_0 the pointer is same as input matrix;
                in case of any other MBConv6_! it's the output of Conv2d 
                and BN with swish
    */
    
    Matrix H_OUT;
    Matrix tmp1; Matrix *ptr_mat; 
 
    if (MBCONV1_0_flag == 1)
      ptr_mat = Input;
    else
    {     
      Set_DeviceMatrix(Input -> height, Input -> width , FD1, 
                       &tmp1,
                       "Output_1 is allocated in device memory"); 
               
      // 1st layer: 1x1 Conv2d, stride = 1, padding = 0, K = 1
      Conv2d_Layer(Input, F1, &tmp1, 1, 0,
                   input_channels, FD1, FD1,
                   Conv2d_1_x_1,
                   NO_ACTIVATION, 0, NULL);
  
      BN_ALL_PRE_DEFINED(&tmp1, SWISH_ACTIVATION, 
                          MBConv_expansion_conv_BN_mean,    MBConv_expansion_conv_BN_variance,
                          MBConv_expansion_conv_BN_weights, MBConv_expansion_conv_BN_bias);
      ptr_mat = &tmp1;
    }

   // 2nd Layer: KxK DWconv, stride = s, padding = p, K = k

    // Height and width changes, Only depth remains still
    int OutputHeight = (ptr_mat -> height + 2 * padding - FilterSizeDW)/Stride + 1;
    int OutputWidth = (ptr_mat -> width + 2 * padding - FilterSizeDW)/Stride + 1;
    int OutputDepth = ptr_mat -> depth;
 
    // Set and allocate tmp2 matrix; it's a transistion between expansion and squeeze
    Matrix tmp2;
    Set_DeviceMatrix(OutputHeight, OutputWidth, OutputDepth, &tmp2,
                    "Output_2 is allocated in device memory");    

    Conv2d_Layer(ptr_mat, F2, &tmp2,
                 Stride, padding, FD1, FD2, FD2, DWConv_k_x_k,
                 NO_ACTIVATION, 0, NULL);
  

    BN_ALL_PRE_DEFINED(&tmp2, SWISH_ACTIVATION, 
                       MBConv_depthwise_conv_BN_mean,     MBConv_depthwise_conv_BN_variance,
                       MBConv_depthwise_conv_BN_weights,  MBConv_depthwise_conv_BN_bias);
  
    // 3rd Layer: squeeze and excitation

    /*
      Squeeze excite layer doesn't change the final output dimensions;
      SE_OUT can be removed; Do so later
    */
 
    Matrix *SE_OUT;
    Squeeze_and_Excite(&tmp2, SE_OUT, F3, F4,
                        FD4, FD3, FD4, FD3,
                        bias1, bias2);

    // 4th Layer: 1x1 Conv2d
    // MBConv output pointer is set and finally updated after this layer execution
    Set_DeviceMatrix(tmp2.height, tmp2.width, FD5, MBConvOut,
                     "Matrix final output is allocated in device memory");
 

    // 1x1 Conv2d layer
    Conv2d_Layer(&tmp2, F5, MBConvOut, 1, 0, FD4, FD5, FD5, Conv2d_1_x_1,
                 NO_ACTIVATION, 0, NULL);


    // BatchNorm layer
    BN_ALL_PRE_DEFINED(MBConvOut, NO_ACTIVATION, 
                       MBConv_project_conv_BN_mean,     MBConv_project_conv_BN_variance,
                       MBConv_project_conv_BN_weights,  MBConv_project_conv_BN_bias);

 
    // Skip identity layer
    if(skip)
    {
      MBConv_SKIP_IDENTITY(MBConvOut, Input);
    }

      
/*    if (MBCONV1_0_flag == 0) 
      cudaFree(tmp1.elements);
    */
    /*
    free(tmp2.elements);
    free(SE_OUT.elements);

    reset_time();
    */
 
    /*
      Always remember to free the input pointer as
      it's a device memory from the previous function
    */
    //cudaFree(Input -> elements);
}


void MBConv_SKIP_IDENTITY(Matrix *parent, Matrix *child)
{
    //printf("The start of Skip connection\n\n");
    
    int nbx = (int)ceil((double)parent -> width / Tile_GEMM);
    int nby = (int)ceil((double)parent -> height / Tile_GEMM);
    int nbz = parent -> depth;

    if (nbx == 0) nbx = 1;

    if (nby == 0) nby = 1;

    // This is the only kernel that runs 3d Grid; 
    // Each block in z dimension controls 1 channel  
    dim3 dim_Grid2(nbx, nby, nbz);
    dim3 dim_Block2(Tile_GEMM, Tile_GEMM, 1);
    
    Identity_Skip <<<dim_Grid2, dim_Block2 >>> (parent -> elements,
                                                  parent -> height,
                                                  parent -> width,
                                                  parent -> depth, 
                                                  child -> elements);
}

void BN_ALL_PRE_DEFINED(Matrix* D_input, int activate, Matrix *mean, Matrix *variance, Matrix *weights, Matrix *bias)
{
    /* The ptr matrix is a device matrix */
 
    //printf("The start of BatchNorm layer\n\n");
    
    /*
      All weights, bias, running mean and running variance
      are pre-defined. Just call the function and use the
      matrices.
      
      All bias, weights, mean and bariance matrices are 1x1xC

      Output Matrix is modified by the equation
      (y = ((x - Mean) / (sqrt(variance) + epsilon)) * weights + bais)
    */

    int nbx = (int)ceil((double)D_input -> width / Tile_GEMM);
    int nby = (int)ceil((double)D_input -> height / Tile_GEMM);
    int nbz = D_input -> depth;

    if (nbx == 0) nbx = 1;
    if (nby == 0) nby = 1;

    // This is the only kernel that runs 3d Grid; 
    // Each block in z dimension controls 1 channel  
    dim3 dim_Grid3(nbx, nby, nbz);
    dim3 dim_Block3(Tile_GEMM, Tile_GEMM, 1);

    BN_Kernel_Final_Layer <<< dim_Grid3, dim_Block3 >>> (D_input -> elements,
                                                         D_input -> height,
                                                         D_input -> width,
                                                         D_input -> depth,
                                                         mean -> elements, variance -> elements,
                                                         weights -> elements, bias -> elements,
                                                         activate);
}

void Padding_Zeros_Function(Matrix* Original_Matrix_Before, int padding_Value, Matrix* padded_Matrix)
{
    /* 
      Note: Matrix coming is a device elemente matrix;
            Original Matrix is a Device input that needs padding
            padded_Matrix is the return of this function;

      Warning: Padded_Matrix has a different size than the Original 
                non padded matrix and it's not allocated in device yet.
                The allocateion is done inside this function.
    */    

    Set_DeviceMatrix(Original_Matrix_Before->height + 2 * padding_Value,
                      Original_Matrix_Before->width + 2 * padding_Value,
                      Original_Matrix_Before->depth,
                      padded_Matrix,
                      "Padded Matrix is allocated in device memory.");

    // 1st: Set padded Matrix with all zeros
    cudaMemset(padded_Matrix -> elements,
               0, padded_Matrix->height * padded_Matrix->width * padded_Matrix->depth * sizeof(float)); 

    int nbx = (int)ceil((double)padded_Matrix -> width / Tile_GEMM);
    int nby = (int)ceil((double)padded_Matrix -> height / Tile_GEMM);
    int nbz = padded_Matrix -> depth;

    if (nbx == 0) nbx = 1;

    if (nby == 0) nby = 1;

    dim3 dim_Grid2(nbx, nby, nbz);
    dim3 dim_Block2(Tile_GEMM, Tile_GEMM, 1);

    // Pass to the copying strided kernel to complete the padding process
 
    Complete_Padding_Process <<< dim_Grid2, dim_Block2 >>> (padded_Matrix -> elements,
                                                            padded_Matrix -> height,
                                                            padded_Matrix -> width,
                                                            padded_Matrix -> depth,
                                                            Original_Matrix_Before -> elements,
                                                            Original_Matrix_Before -> height,
                                                            Original_Matrix_Before -> width,
                                                            Original_Matrix_Before -> depth,
                                                            padding_Value);
}


// Call this function directly for 1x1 conv2d. Don't call for DWConv
void Conv_vidMultiplier(Matrix* out_11, Matrix* D_2, Matrix* D_1,
                        int ReconstructOutHieght, int ReconstructOutWidth, int ReconstructOutDepth,
                        int ConvType, int stride_DW, int activation_type, int BIASED_CHOISE, Matrix *biasMat)
{
    /* Note: Out_11, XXX_Trans and Host_Conv_Filter are device matrices */
 
    // The multiplication kernel is used for the 1x1 Conv2d and kxk Conv2d
    if (ConvType == Conv2d_1_x_1 || ConvType == Regular_Conv)
    {    
        // Get number of blocks
        int nbx = (int)ceil((double)out_11 -> width / (THREAD_GRANULARITY_BLOCKS * Tile_GEMM));
        int nby = (int)ceil((double)out_11 -> height / Tile_GEMM);
        int num_block_for_phases = (int)ceil((double)D_1 -> width / Tile_GEMM);

        // Check for zero blocks to make sure code runs correctly
        if (nbx == 0) nbx = 1;
        if (nby == 0) nby = 1;

        dim3 dim_Grid2(nbx, nby, 1);
        dim3 dim_Block2(Tile_GEMM, Tile_GEMM, 1);
     
        if (BIASED_CHOISE == BIASED)
        {
          Set_HostMatrix(out_11 -> height, 1, 1, biasMat);

          // Call shared memory tiled Multiplication  algorithm

          MatrixMulKernel <<< dim_Grid2, dim_Block2 >>> (D_1 -> elements, D_1 -> height, D_1 -> width, D_1 -> depth,
                                                         D_2 -> elements, D_2 -> height, D_2 -> width, D_2 -> depth,
                                                         out_11 -> elements, out_11 -> height, out_11 -> width, out_11 -> depth,
                                                         num_block_for_phases, activation_type,
                                                         BIASED_CHOISE, biasMat -> elements);         
        }
        else
        {
            

          MatrixMulKernel <<< dim_Grid2, dim_Block2 >>> (D_1 -> elements, D_1 -> height, D_1 -> width, D_1 -> depth,
                                                         D_2 -> elements, D_2 -> height, D_2 -> width, D_2 -> depth,
                                                         out_11 -> elements, out_11 -> height, out_11 -> width, out_11 -> depth,
                                                         num_block_for_phases, activation_type,
                                                         BIASED_CHOISE, NULL);        
        }    
    }

    // This case is for DWConv2d
    else
    {
        int nbx = (int)ceil((double)out_11 -> width / TileDW);
        int nby = (int)ceil((double)out_11 -> height / TileDW);
        int nbz = out_11 -> depth;
     
        if (nbx == 0) nbx = 1;
        if (nby == 0) nby = 1;

        // This is the only kernel that runs 3d Grid; 
        // Each block in z dimension controls 1 channel  
        dim3 dim_Grid2(nbx, nby, nbz);
        dim3 dim_Block2(TileDW, TileDW, 1);


        DWConv2d_kernel << < dim_Grid2, dim_Block2 >> > (D_2 -> elements, D_2 -> height, D_2 -> width, D_2 -> depth,
                                                         D_1 -> elements, D_1 -> height, D_1 -> width, D_1 -> depth,
                                                         out_11 -> elements, out_11 -> height, out_11 -> width, out_11 -> depth,
                                                         stride_DW);
    }
 
    // Reset the output dimensions to continue in the network
    Set_HostMatrix(ReconstructOutHieght, ReconstructOutWidth, ReconstructOutDepth, out_11);
}

void Input_Unroll_gpu(int st_stride, Matrix* Device_Input, Matrix* Device_Unrolled, int O_H, int O_W, int Filter_Size)
{   
    /* Note: All the function input matrices are device matrices.
            Device_Input matrix is already allocated and ready.
            Device_Unrolled matrix is already allocated and ready. 
    */
    
    int nbx = (int)ceil((double)O_W / TileDW);
    int nby = (int)ceil((double)O_H / TileDW);
    int nbz = Device_Input -> depth;

    if (nbx == 0) nbx = 1;

    if (nby == 0) nby = 1;
 
    dim3 dim_Grid2(nbx, nby, nbz);
    dim3 dim_Block2(TileDW, TileDW, 1);

    // You need to use cudaDeviceSynchronize if the kernel isn't working

    INPUT_UNROLLING <<< dim_Grid2, dim_Block2 >>> (st_stride, Filter_Size,
                                                   
                                                   Device_Input -> elements,
                                                   Device_Input -> height,
                                                   Device_Input -> width,
                                                   Device_Input -> depth,

                                                   Device_Unrolled -> elements,
                                                   Device_Unrolled -> height,
                                                   Device_Unrolled -> width,
                                                   Device_Unrolled -> depth,

                                                   O_H, O_W);

    
    //cudaDeviceSynchronize(); 

    cudaError err = cudaGetLastError();

    if ( err != cudaSuccess )
    {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
      exit(-1);
    } 
}

void DEFINE_FILTERS_FOR_MBCONV_BN(  Matrix *EXP_MEAN, 		  float *filter1, int size_1,
                                    Matrix *EXP_VARIANCE, 	float *filter2, int size_2,
                                    Matrix *EXP_WEIGHTS, 	  float *filter3, int size_3,
                                    Matrix *EXP_BIAS, 		  float *filter4, int size_4,
                                  
                                    Matrix *DW_MEAN, 		    float *filter5, int size_5,
                                    Matrix *DW_VARIANCE, 	  float *filter6, int size_6,
                                    Matrix *DW_WEIGHTS, 		float *filter7, int size_7,
                                    Matrix *DW_BIAS, 		    float *filter8, int size_8,
                                    
                                    Matrix *PRJ_MEAN, 		  float *filter9,  int size_9,
                                    Matrix *PRJ_VARIANCE, 	float *filter10, int size_10,
                                    Matrix *PRJ_WEIGHTS, 	  float *filter11, int size_11,
                                    Matrix *PRJ_BIAS, 		  float *filter12, int size_12)
{
  if (MBCONV1_0_flag);
  else
  {
    set_allocate_copy_array_Device(EXP_MEAN, filter1,
                      size_1, 1, 1,
                      "expand mean"); 
    set_allocate_copy_array_Device(EXP_VARIANCE, filter2,
                      size_2, 1, 1,
                      "expand variance"); 
    set_allocate_copy_array_Device(EXP_WEIGHTS, filter3,
                      size_3, 1, 1,
                      "expand weights"); 
    set_allocate_copy_array_Device(EXP_BIAS, filter4,
                      size_4, 1, 1,
                      "expand bias");
  } 
									  
	set_allocate_copy_array_Device(DW_MEAN, filter5,
									  size_5, 1, 1,
									  "DW mean"); 
	set_allocate_copy_array_Device(DW_VARIANCE, filter6,
									  size_6, 1, 1,
									  "DW variance"); 
	set_allocate_copy_array_Device(DW_WEIGHTS, filter7,
									  size_7, 1, 1,
									  "DW weights"); 
	set_allocate_copy_array_Device(DW_BIAS, filter8,
									  size_8, 1, 1,
									  "expand bias"); 

	set_allocate_copy_array_Device(PRJ_MEAN, filter9,
									  size_9, 1, 1,
									  "DW mean"); 
	set_allocate_copy_array_Device(PRJ_VARIANCE, filter10,
									  size_10, 1, 1,
									  "DW variance"); 
	set_allocate_copy_array_Device(PRJ_WEIGHTS, filter11,
									  size_11, 1, 1,
									  "DW weights"); 
	set_allocate_copy_array_Device(PRJ_BIAS, filter12,
									  size_12, 1, 1,
									  "expand bias"); 
}

// 3 Sequential Operations: Same as "set_allocate_copy_Matrix_Device",
// However, it uses a pointer to float as a parent.
void set_allocate_copy_array_Device(Matrix *child, float *parent,
									int height, int width, int depth,
									char *notification)
{
	Set_DeviceMatrix(height, width, depth, child, notification);

	size_t size = height * width * depth * sizeof(float);
 
	cudaError err = cudaMemcpy(child -> elements, parent, size,
								cudaMemcpyHostToDevice);
  
	CheckCudaError(notification, err);
}

// 3 Sequential Operations: Set dimensions, allocate device memory and copy.
void set_allocate_copy_Matrix_Device(Matrix *child, Matrix *parent, char *notification)
{
	Set_DeviceMatrix(parent -> height, parent -> width, parent -> depth,
					          child, notification);

	size_t size = parent -> height * parent -> width * parent -> depth * sizeof(float);
	
	cudaError err = cudaMemcpy(child -> elements, parent -> elements,
								              size, cudaMemcpyHostToDevice);
	CheckCudaError(notification, err);
}

void set_allocate_copy_Matrix_Device_specific(Matrix *child, Matrix *parent, char *notification, int height, int width, int depth)
{
	Set_DeviceMatrix(height, width, depth, child, notification);

	size_t size = child -> height * child -> width * child -> depth * sizeof(float);
	

	cudaError err = cudaMemcpy(child -> elements, parent -> elements,
								            size, cudaMemcpyHostToDevice);

  
	CheckCudaError(notification, err);
}

void just_copy_HTD(Matrix *child, Matrix *parent, char *notification)
{
    // Read C from device memory
  size_t size = parent -> width * parent -> height * parent -> depth * sizeof(float);
    
	cudaError err = cudaMemcpy(child -> elements, parent -> elements, size, cudaMemcpyHostToDevice);

  
	CheckCudaError(notification, err);
}

void just_copy_DTH(Matrix *child, Matrix *parent, char *notification)
{
  // Read C from device memory
  size_t size = parent -> width * parent -> height * parent -> depth * sizeof(float);
  

	cudaError err = cudaMemcpy(child -> elements, parent -> elements, size, cudaMemcpyDeviceToHost);
  
	CheckCudaError(notification, err);
}

void set_allocate_Host(Matrix *ptr, int height, int width, int depth)
{
	// Note this function allocates memory, remember to free 
	Set_HostMatrix(height, width, depth, ptr);
	
	int Fsize = height * width * depth* sizeof(float);
 
	ptr -> elements = (float *) malloc(Fsize);
}

void FreeHost_Allocated(Matrix *ptr)
{
	free(ptr -> elements);
}

// Allocations for Device matrices
void Set_DeviceMatrix(int height, int width, int depth, Matrix* ptr, char* NamePtr)
{
    ptr->width = width;
    ptr->height = height;
    ptr->depth = depth;

    size_t size = width * height * depth * sizeof(float);
    cudaError err = cudaMalloc((void **)&(ptr->elements), size);
    CheckCudaError(NamePtr, err);
}

void Set_HostMatrix(int height, int width, int depth, Matrix* ptr)
{
    ptr->width = width;
    ptr->height = height;
    ptr->depth = depth;
}

void CheckCudaError(char* ptr, cudaError err)
{
    if (err == cudaSuccess);
    else
        printf("CUDA error in %s: %s\n", ptr, cudaGetErrorString(err));
}


void show_me_enhanced(Matrix* ptr, char* NamePtr)
{
    if(show_out == 1)
    {
      setvbuf(stdout, NULL, _IOLBF, 0);

          printf("%s,"
              "it has height = %d, "
              "width = %d, "
              "depth = %d \n",
              NamePtr, ptr->height, ptr->width, ptr->depth);

          printf("{\n");
          for (int i = 0; i < ptr -> height * ptr -> width * ptr -> depth; i++)
          {
              if (i % ptr->width == 0 && i >= ptr->width)
                  printf("\n");

              if (i % (ptr->width * ptr->height) == 0 && i >= (ptr->width * ptr->height));
                  //printf("\n");

              printf("%.8f", ptr->elements[i]);
              if (i + 1 == ptr->height * ptr->width * ptr->depth);
              else
                  printf(", ");
          }

          printf("} \n");
          printf("\n");

          setvbuf(stdout, NULL, _IOLBF, 0);        
    }
}


void start()
{
  HANDLE_ERROR(cudaEventCreate(&start_timing));
  HANDLE_ERROR(cudaEventCreate(&stop_timing));
  HANDLE_ERROR(cudaEventRecord(start_timing, 0));
}

void stop(char *notification, int pause_time)
{
  HANDLE_ERROR(cudaEventRecord(stop_timing, 0));
  HANDLE_ERROR(cudaEventSynchronize(stop_timing));
  HANDLE_ERROR(cudaEventElapsedTime(&time_defined, start_timing, stop_timing));
 
  if(pause_time)
 {
    tmp_time += time_defined; 
 }   
 
  else
  {
    tmp_time = 0;
    printf("Time elapsed for %s:  %.8f ms\n", notification, time_defined);  
    total_time_for_layer += time_defined;
  }
}

void after_pause(char *notification)
{
  printf("Time elapsed for %s: %.8f ms\n", notification, tmp_time); 
  total_time_for_layer += tmp_time;
 
  tmp_time = 0;         
}

void reset_time()
{
  printf("Total time: %.8f ms\n", total_time_for_layer); 
  total_time_for_layer = 0;
}

void show_me_enhanced_from_devince(Matrix *ptr, char *notification)
{
    Matrix H_OUT;

    set_allocate_Host(&H_OUT, ptr -> height, ptr -> width, ptr -> depth);

    just_copy_DTH(&H_OUT, ptr, "show_device_elements");
  
    show_out = 1;
    show_me_enhanced(&H_OUT, notification);
    show_out = 0;  
}

'File written in /content/src/FUNCTIONS.cu'

In [82]:
# !nvcc -o /content/src/my_curand /content/src/my_curand.cu -lcurand --use_fast_math
!nvcc -o /content/src/EfficientNetB0 /content/src/APP.cu /content/src/kernels.cu /content/src/FUNCTIONS.cu --use_fast_math

/content/src/APP.cu(110): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(116): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(384): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(388): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(393): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(396): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(399): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(402): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(405): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(408): warning: conversion from a string literal to "char *" is deprecated

/content/src/APP.cu(411): warning: conversion from

In [97]:
!/content/src/EfficientNetB0

Time elapsed for Model: :  10.23910427 ms


In [ ]:
with open('Results'+'.txt', 'w') as out:
    out.write(cap.stdout)

# !nvidia-smi

In [ ]:
!cp Results.txt /content/drive/MyDrive/Colab_Notebooks/